### 단일모델 결과

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# ── [로그 무음 설정: ultralytics 임포트 전에] ───────────────────────────
import os, logging
os.environ["ULTRALYTICS_QUIET"] = "1"   # 내부 로그/이미지별 출력 억제
# os.environ["RICH_PROGRESS_BAR"] = "0"  # 진행바도 끄고 싶으면 주석 해제

# ==============================
# YOLOv8 Test Metrics (TensorRT) — mAP == model.val(conf=0.001), AUROC 포함
# + (옵션) 단일 모델 예측 박스 시각화/저장
# + (옵션) Divot 전용 ROC 테이블/곡선 저장
# ==============================
from ultralytics import YOLO
from ultralytics.utils import LOGGER
from ultralytics.utils.metrics import ap_per_class, box_iou
from pathlib import Path
from tqdm import tqdm
import numpy as np
import torch
import yaml
import math
from sklearn.metrics import roc_auc_score, roc_curve, auc
from datetime import datetime
import cv2
import random
import matplotlib.pyplot as plt
import csv

# ── [로거 레벨 하향] ───────────────────────────────────────────────────
LOGGER.setLevel(logging.ERROR)

# ==============================
# 설정
# ==============================

ENGINE    = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251017/20251017_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_25_20251024_add/weights/best_fp16_bs1_640px_static.engine"
#ENGINE    = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251124/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_25_20251122/weights/best_fp16_bs1_640px_static.engine"
DATA_YAML    = "/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251113/data.yaml"
TEST_DIR     = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251113/images/test")
GT_LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251113/labels/test")

IMG_SIZE     = 640
DEVICE       = 0
PRED_CONF    = 0.001   # ← AUROC/mAP 정석: 낮게 저장(0~0.001). 시각화 임계값은 아래에서 별도 사용 권장
MATCH_IOU    = 0.5     # 매칭 IoU (mAP/AUROC 공통), mAP clone 비교는 '>' 사용
IOU_THRS     = np.linspace(0.5, 0.95, 10, dtype=np.float32)
AUGMENT      = False
AGNOSTIC_NMS = True
RECT         = False

# ===[시각화 토글/옵션]=========================================
DRAW_SAVE           = False      # ← True면 원본 이미지에 예측 박스를 그려 저장
DRAW_USE_PRED_CONF  = False      # ← 시각화 임계값으로 PRED_CONF 사용 여부
DRAW_SCORE_MIN      = 0.5        # ← 시각화 전용 임계값(권장: 운영값)
DRAW_OUTDIR_NAME    = "viz"      # 출력 서브폴더명
# 박스 스타일
DRAW_THICK_BASE     = 2          # 기본 두께(이미지 크기에 비례하여 가중)
DRAW_FONT_SCALE     = 0.5
DRAW_TEXT_THICK     = 2

# ===[Divot 전용 ROC 테이블/그래프]===============================
DRAW_DIVOT_ROC     = True
TARGET_CLASS_NAME  = "Divot"
# 기존: ROC_THRESH_GRID = np.linspace(0.0, 1.0, 11)
# 수정 ↓↓↓
ROC_THRESH_GRID    = np.concatenate(([0.001], np.linspace(0.0, 1.0, 11)))
ROC_OUTDIR_NAME    = "roc_divot"
# ================================================================

# ==============================
# 모델/클래스 로드
# ==============================
model = YOLO(ENGINE)
with open(DATA_YAML, "r") as f:
    data_cfg = yaml.safe_load(f)
CLASS_NAMES = data_cfg["names"]
NUM_CLASSES = len(CLASS_NAMES)

# ==============================
# 예측 (매 실행마다 새 폴더)
# ==============================
project_root = Path("/home/dw/ws_job_msislab/Golf_Project/runs/predict_eval/val_clone")
project_root.mkdir(parents=True, exist_ok=True)
run_name = f"pred_labels_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}"
print(f"🚀 Running TensorRT inference (conf={PRED_CONF}) -> {project_root}/{run_name}")

model.predict(
    source=str(TEST_DIR),
    imgsz=IMG_SIZE,
    conf=PRED_CONF,
    iou=MATCH_IOU,                 # NMS IoU
    device=DEVICE,
    rect=RECT,
    agnostic_nms=AGNOSTIC_NMS,
    augment=AUGMENT,
    deterministic=True,
    seed=1,
    save_txt=True,
    save_conf=True,
    project=project_root,          # 고정 project
    name=run_name,                 # 매 실행 다른 name
    exist_ok=False,                # 동일 이름 있으면 실패 → 새 폴더 강제
    verbose=False,                 # ← per-image 로그 차단
)

PRED_LABEL_DIR = project_root / run_name / "labels"
print(f"✅ Predictions saved to: {PRED_LABEL_DIR}")

# ==============================
# 유틸
# ==============================
def load_yolo_txt(path: Path):
    if (path is None) or (not path.exists()):
        return np.zeros((0, 6), dtype=np.float32)
    rows = []
    with open(path, "r") as f:
        for line in f:
            vals = list(map(float, line.strip().split()))
            if len(vals) == 5:
                c, x, y, w, h = vals; conf = 1.0  # GT
            else:
                c, x, y, w, h, conf = vals       # Pred
            rows.append([c, x, y, w, h, conf])
    return np.asarray(rows, dtype=np.float32)

def xywhn2xyxy_pixels(xywhn, W, H):
    """정규화 xywh → 픽셀 단위 xyxy"""
    if len(xywhn) == 0:
        return np.zeros((0, 4), dtype=np.float32)
    x, y, w, h = xywhn.T
    x1 = (x - w/2) * W
    y1 = (y - h/2) * H
    x2 = (x + w/2) * W
    y2 = (y + h/2) * H
    return np.stack([x1, y1, x2, y2], axis=1).astype(np.float32)

def stable_conf_sort(conf: np.ndarray):
    idx = np.arange(len(conf))
    return np.lexsort((idx, -conf))

def cat_or_empty(arrs, axis=0, empty_shape=None, dtype=None):
    arrs = [a for a in arrs if a is not None]
    if len(arrs) == 0:
        return np.zeros(empty_shape, dtype=dtype)
    try:
        return np.concatenate(arrs, axis=axis)
    except Exception:
        return np.zeros(empty_shape, dtype=dtype)

# ==============================
# 평가 준비 (GT 기준)
# ==============================
gt_files = sorted(GT_LABEL_DIR.glob("*.txt"))
pred_map = {p.stem: p for p in PRED_LABEL_DIR.glob("*.txt")}

# ==============================
# mAP (val@0.001 clone: conf 컷 없음)
# ==============================
tp_list, conf_list, pred_cls_list, target_cls_list = [], [], [], []
for gf in tqdm(gt_files, desc="Collecting stats for mAP (GT-based)..."):
    stem = gf.stem
    pf = pred_map.get(stem, None)
    preds = load_yolo_txt(pf)
    gts   = load_yolo_txt(gf)

    gt_boxes   = xywhn2xyxy_pixels(gts[:, 1:5], W=1920, H=1536) if len(gts) else np.zeros((0,4), dtype=np.float32)
    gt_cls     = gts[:, 0].astype(int)  if len(gts) else np.zeros((0,), dtype=int)
    pred_boxes = xywhn2xyxy_pixels(preds[:, 1:5], W=1920, H=1536) if len(preds) else np.zeros((0,4), dtype=np.float32)
    pred_cls   = preds[:, 0].astype(int) if len(preds) else np.zeros((0,), dtype=int)
    conf       = preds[:, 5].astype(np.float32) if len(preds) else np.zeros((0,), dtype=np.float32)

    correct = np.zeros((len(preds), len(IOU_THRS)), dtype=bool)
    if len(preds) and len(gts):
        ious = box_iou(torch.tensor(pred_boxes, dtype=torch.float32),
                       torch.tensor(gt_boxes,   dtype=torch.float32)).numpy()
        order = stable_conf_sort(conf)
        pred_boxes, pred_cls, conf = pred_boxes[order], pred_cls[order], conf[order]
        ious = ious[order]
        for j, thr in enumerate(IOU_THRS):
            detected = []
            for k, pc in enumerate(pred_cls):
                best_iou, best_gt = 0.0, -1
                for gi, gc in enumerate(gt_cls):
                    if gc != pc or gi in detected:
                        continue
                    iou_val = ious[k, gi]
                    if iou_val > best_iou:
                        best_iou, best_gt = iou_val, gi
                if best_iou > thr:   # YOLO val과 동일: >
                    correct[k, j] = True
                    detected.append(best_gt)

    tp_list.append(correct)
    conf_list.append(conf)
    pred_cls_list.append(pred_cls)
    target_cls_list.append(gt_cls)

tp             = cat_or_empty(tp_list, axis=0, empty_shape=(0, len(IOU_THRS)), dtype=bool)
conf_all       = cat_or_empty(conf_list, axis=0, empty_shape=(0,), dtype=np.float32)
pred_cls_all   = cat_or_empty(pred_cls_list, axis=0, empty_shape=(0,), dtype=int)
target_cls_all = cat_or_empty(target_cls_list, axis=0, empty_shape=(0,), dtype=int)

tp_c, fp_c, p, r, f1, ap, ap_class, *_ = ap_per_class(tp, conf_all, pred_cls_all, target_cls_all)

# ---- 안전 캐스팅 & 요약값 계산 (프린트 전에 실행) ----
p_arr  = np.asarray(p,  dtype=float).reshape(-1)
r_arr  = np.asarray(r,  dtype=float).reshape(-1)
ap_np  = np.asarray(ap, dtype=float)

# ap 모양 방어: (n_class, n_iou) 가 아닐 때 정리
if ap_np.ndim == 1:
    n_iou = len(IOU_THRS)
    assert ap_np.size % n_iou == 0, "ap shape unexpected; cannot reshape"
    ap_np = ap_np.reshape(-1, n_iou)

p_mean   = float(p_arr.mean()) if p_arr.size else 0.0
r_mean   = float(r_arr.mean()) if r_arr.size else 0.0
mAP50    = float(ap_np[:, 0].mean()) if ap_np.size else 0.0
mAP5095  = float(ap_np.mean())       if ap_np.size else 0.0

# per-class dict (프린트 루프에서 사용)
ap_dict = {
    int(c): (float(p_arr[i]) if i < p_arr.size else 0.0,
             float(r_arr[i]) if i < r_arr.size else 0.0,
             float(ap_np[i, 0]) if i < ap_np.shape[0] else 0.0,
             float(ap_np[i].mean()) if i < ap_np.shape[0] else 0.0)
    for i, c in enumerate(ap_class)
}

# ==============================
# AUROC + TP/FP/FN/GT  (FN을 점수 0 양성으로 반영)
# ==============================
per_true  = {i: [] for i in range(NUM_CLASSES)}
per_score = {i: [] for i in range(NUM_CLASSES)}
tp_cnt = np.zeros(NUM_CLASSES, dtype=int)
fp_cnt = np.zeros(NUM_CLASSES, dtype=int)
fn_cnt = np.zeros(NUM_CLASSES, dtype=int)
gt_cnt = np.zeros(NUM_CLASSES, dtype=int)

for gf in tqdm(gt_files, desc="Matching for AUROC (GT-based)..."):
    stem = gf.stem
    pf = pred_map.get(stem, None)
    preds = load_yolo_txt(pf)
    gts   = load_yolo_txt(gf)

    if len(gts):
        for c in gts[:, 0].astype(int):
            gt_cnt[c] += 1

    # 예측 없음 + GT 존재 → 전부 FN (점수 0의 양성으로 추가)
    if len(preds) == 0 and len(gts) > 0:
        for c in gts[:, 0].astype(int):
            fn_cnt[c] += 1
            per_true[c].append(1)     # 양성
            per_score[c].append(0.0)  # 미검출 → 점수 0
        continue

    gt_boxes   = xywhn2xyxy_pixels(gts[:, 1:5], W=1920, H=1536) if len(gts) else np.zeros((0,4), dtype=np.float32)
    gt_cls     = gts[:, 0].astype(int)  if len(gts) else np.zeros((0,), dtype=int)
    pred_boxes = xywhn2xyxy_pixels(preds[:, 1:5], W=1920, H=1536) if len(preds) else np.zeros((0,4), dtype=np.float32)
    pred_cls   = preds[:, 0].astype(int) if len(preds) else np.zeros((0,), dtype=int)
    conf       = preds[:, 5].astype(np.float32) if len(preds) else np.zeros((0,), dtype=np.float32)

    if len(preds) and len(gts):
        ious = box_iou(torch.tensor(pred_boxes, dtype=torch.float32),
                       torch.tensor(gt_boxes,   dtype=torch.float32)).numpy()
        order = stable_conf_sort(conf)
        pred_boxes, pred_cls, conf = pred_boxes[order], pred_cls[order], conf[order]
        ious = ious[order]

        used = set()
        for k, (pc, cf) in enumerate(zip(pred_cls, conf)):
            best_iou, best_gt = 0.0, -1
            for gi, gc in enumerate(gt_cls):
                if gc != pc or gi in used:
                    continue
                iou_val = ious[k, gi]
                if iou_val > best_iou:
                    best_iou, best_gt = iou_val, gi
            if best_iou > MATCH_IOU and best_gt != -1:      # '>' 로 통일
                tp_cnt[pc] += 1
                per_true[pc].append(1)
                per_score[pc].append(cf)
                used.add(best_gt)
            else:
                fp_cnt[pc] += 1
                per_true[pc].append(0)
                per_score[pc].append(cf)

        # 매칭되지 않은 GT → FN을 점수 0 양성으로 추가
        for gi, gc in enumerate(gt_cls):
            if gi not in used:
                fn_cnt[gc] += 1
                per_true[gc].append(1)
                per_score[gc].append(0.0)
    elif len(preds) > 0 and len(gts) == 0:
        for c, cf in zip(pred_cls, conf):
            fp_cnt[c] += 1
            per_true[c].append(0)
            per_score[c].append(cf)

aurocs = []
for c in range(NUM_CLASSES):
    y_true  = np.asarray(per_true[c], dtype=np.float32)
    y_score = np.asarray(per_score[c], dtype=np.float32)
    if len(y_true) < 2 or len(np.unique(y_true)) < 2:
        aurocs.append(float("nan"))
    else:
        try:
            aurocs.append(roc_auc_score(y_true, y_score))
        except ValueError:
            aurocs.append(float("nan"))

mean_auroc = np.nanmean(aurocs) if np.any(~np.isnan(aurocs)) else float("nan")

# ==============================
# (NEW) Divot ROC 테이블 & 곡선 (토글)
# ==============================
if DRAW_DIVOT_ROC:
    try:
        CID = CLASS_NAMES.index(TARGET_CLASS_NAME)
    except ValueError:
        CID = None

    if CID is None:
        print(f"[ROC] 클래스 '{TARGET_CLASS_NAME}'를 names에서 찾지 못했습니다. 스킵합니다.")
    else:
        y_true_div  = np.asarray(per_true[CID], dtype=np.int32)
        y_score_div = np.asarray(per_score[CID], dtype=np.float32)

        if y_true_div.size == 0 or len(np.unique(y_true_div)) < 2:
            print(f"[ROC] '{TARGET_CLASS_NAME}'에 유효한 양성/음성이 부족합니다. 스킵합니다.")
        else:
            roc_dir = (project_root / run_name / ROC_OUTDIR_NAME)
            roc_dir.mkdir(parents=True, exist_ok=True)

            # 1) 임의 threshold 그리드로 TP/FP/TN/FN/TPR/FPR 표 계산
            rows = []
            P = int((y_true_div == 1).sum())
            N = int((y_true_div == 0).sum())
            for thr in ROC_THRESH_GRID:
                y_pred = (y_score_div >= thr).astype(np.int32)
                TP = int(((y_pred == 1) & (y_true_div == 1)).sum())
                FP = int(((y_pred == 1) & (y_true_div == 0)).sum())
                TN = int(((y_pred == 0) & (y_true_div == 0)).sum())
                FN = int(((y_pred == 0) & (y_true_div == 1)).sum())
                TPR = (TP / P) if P > 0 else 0.0
                FPR = (FP / N) if N > 0 else 0.0
                rows.append([thr, TP, FP, TN, FN, TPR, FPR])
            rows.sort(key=lambda x: x[0])

            csv_path = roc_dir / "divot_roc_table.csv"
            with open(csv_path, "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(["threshold", "TP", "FP", "TN", "FN", "TPR", "FPR"])
                writer.writerows(rows)
            print(f"[ROC] Divot ROC table saved: {csv_path}")

            # 콘솔에도 간단 프리뷰
            print("\n[ROC] Divot — threshold sweep preview")
            print("thr     TP    FP    TN    FN    TPR      FPR")
            for r in rows[:11]:
                print(f"{r[0]:.3f}  {r[1]:5d} {r[2]:5d} {r[3]:5d} {r[4]:5d}  {r[5]:.4f}  {r[6]:.4f}")

            # 2) sklearn으로 ROC 곡선 본판 계산/그리기 (모든 점수 유니크 임계값)
            fpr, tpr, thr = roc_curve(y_true_div, y_score_div)
            roc_auc = auc(fpr, tpr)

            plt.figure(figsize=(5,5))
            plt.plot(fpr, tpr, marker='.', linewidth=1, label=f'ROC (AUC={roc_auc:.4f})')
            plt.plot([0,1],[0,1],'--', linewidth=1, label='chance')
            plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title(f'ROC — {TARGET_CLASS_NAME}')
            plt.legend()
            plt.grid(True, alpha=0.3)
            fig_path = roc_dir / "divot_roc_curve.png"
            plt.savefig(fig_path, bbox_inches="tight", dpi=150)
            plt.close()
            print(f"[ROC] Divot ROC curve saved: {fig_path}")

# ==============================
# 출력 (mAP/AUROC)
# ==============================
print(f"\n================= ✅ MANUAL EVALUATION (mAP clone of val@conf={PRED_CONF:.3f}) =================")
print(f"{'all':<15}  mP={p_mean:.3f}  mR={r_mean:.3f}  mAP50={mAP50:.3f}  mAP50-95={mAP5095:.3f}")
for i in range(NUM_CLASSES):
    if i in ap_dict:
        pi, ri, ap50_i, ap5095_i = ap_dict[i]
    else:
        pi, ri, ap50_i, ap5095_i = 0.0, 0.0, 0.0, 0.0
    print(f"{CLASS_NAMES[i]:<15}  P={pi:.3f}  R={ri:.3f}  mAP50={ap50_i:.3f}  mAP50-95={ap5095_i:.3f}")
print("======================================================================")

print("\n📈 AUROC Evaluation per class")
print("─────────────────────────────────────────────────────────────")
for i in range(NUM_CLASSES):
    auc_str = f"{aurocs[i]:.4f}" if not math.isnan(aurocs[i]) else "NaN"
    print(f"{CLASS_NAMES[i]:<18s}: AUROC={auc_str:<8s} | "
          f"TP={tp_cnt[i]:<4d} FP={fp_cnt[i]:<4d} FN={fn_cnt[i]:<4d} GT={gt_cnt[i]:<4d}")
print("─────────────────────────────────────────────────────────────")
print(f"🔹 Mean AUROC (macro): {mean_auroc:.4f}" if not math.isnan(mean_auroc) else "🔹 Mean AUROC: NaN")
print("======================================================================")

# ==============================
# (옵션) 시각화/저장
# ==============================
def _class_colors(n):
    random.seed(0)
    colors = []
    for i in range(n):
        # 고정된 파렛트 생성
        random.seed(i + 12345)
        colors.append((random.randint(60,255), random.randint(60,255), random.randint(60,255)))
    return colors

if DRAW_SAVE:
    viz_dir = project_root / run_name / DRAW_OUTDIR_NAME
    viz_dir.mkdir(parents=True, exist_ok=True)
    colors = _class_colors(NUM_CLASSES)

    img_paths = [p for p in sorted(TEST_DIR.glob("*")) if p.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}]
    thr = PRED_CONF if DRAW_USE_PRED_CONF else DRAW_SCORE_MIN

    print(f"\n🖼  Drawing predictions to: {viz_dir}  (thr={thr})")
    for img_path in tqdm(img_paths, desc="Drawing"):
        img = cv2.imread(str(img_path))
        if img is None:
            continue
        H, W = img.shape[:2]
        thick = max(1, int(DRAW_THICK_BASE * (W + H) / 2000))  # 이미지 크기에 비례

        stem = img_path.stem
        pred_txt = PRED_LABEL_DIR / f"{stem}.txt"
        preds = load_yolo_txt(pred_txt)

        if len(preds):
            # 재필터 (predict 저장 conf와 무관하게 시각화 임계값으로 필터)
            keep = preds[:,5] >= thr
            preds = preds[keep]

            if len(preds):
                boxes_xyxy = xywhn2xyxy_pixels(preds[:,1:5], W=W, H=H)
                cls_ids = preds[:,0].astype(int)
                confs = preds[:,5].astype(float)

                for (x1,y1,x2,y2), cid, cf in zip(boxes_xyxy, cls_ids, confs):
                    c = colors[cid % len(colors)]
                    cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), c, thickness=thick)
                    label = f"{CLASS_NAMES[cid]} {cf:.2f}"
                    (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, DRAW_FONT_SCALE, DRAW_TEXT_THICK)
                    # 배경 상자
                    cv2.rectangle(img, (int(x1), int(y1)-th-4), (int(x1)+tw+4, int(y1)), c, -1)
                    cv2.putText(img, label, (int(x1)+2, int(y1)-2),
                                cv2.FONT_HERSHEY_SIMPLEX, DRAW_FONT_SCALE, (0,0,0), DRAW_TEXT_THICK, cv2.LINE_AA)

        out_path = viz_dir / img_path.name
        cv2.imwrite(str(out_path), img)

print("\n✅ Done.")


### 단일모델 viz수정버전

In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# ── [로그 무음 설정: ultralytics 임포트 전에] ───────────────────────────
import os, logging
os.environ["ULTRALYTICS_QUIET"] = "1"   # 내부 로그/이미지별 출력 억제
# os.environ["RICH_PROGRESS_BAR"] = "0"  # 진행바도 끄고 싶으면 주석 해제

# ==============================
# YOLOv8 Test Metrics (TensorRT) — mAP == model.val(conf=0.001), AUROC 포함
# + (옵션) 단일 모델 예측 박스 시각화/저장
# + (옵션) Divot 전용 ROC 테이블/곡선 저장
# ==============================
from ultralytics import YOLO
from ultralytics.utils import LOGGER
from ultralytics.utils.metrics import ap_per_class, box_iou
from pathlib import Path
from tqdm import tqdm
import numpy as np
import torch
import yaml
import math
from sklearn.metrics import roc_auc_score, roc_curve, auc
from datetime import datetime
import cv2
import random
import matplotlib.pyplot as plt
import csv

# ── [로거 레벨 하향] ───────────────────────────────────────────────────
LOGGER.setLevel(logging.ERROR)

# ==============================
# 설정
# ==============================
#ENGINE       = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251124/20251107_data_yolov8s_img640_SGD_cls1.0_box7.5_dfl1.5_rectFalse_seed_57_20251123/weights/best_fp16_bs1_640px_static.engine"
ENGINE    = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251124/20251107_data_yolov8s_img640_SGD_cls1.0_box7.5_dfl1.5_rectFalse_seed_25_20251123/weights/best_fp16_bs1_640px_static.engine"
DATA_YAML    = "/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251113_a/data.yaml"
TEST_DIR     = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251113_a/images/test")
GT_LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251113_a/labels/test")

IMG_SIZE     = 640
DEVICE       = 0
PRED_CONF    = 0.001   # ← AUROC/mAP 정석: 낮게 저장(0~0.001). 시각화 임계값은 아래에서 별도 사용 권장
MATCH_IOU    = 0.5     # 매칭 IoU (mAP/AUROC 공통), mAP clone 비교는 '>' 사용
IOU_THRS     = np.linspace(0.5, 0.95, 10, dtype=np.float32)
AUGMENT      = False
AGNOSTIC_NMS = True
RECT         = False

# ===[시각화 토글/옵션]=========================================
DRAW_SAVE           = True     # ← True면 원본 이미지에 예측 박스를 그려 저장
DRAW_USE_PRED_CONF  = False      # ← 시각화 임계값으로 PRED_CONF 사용 여부
DRAW_SCORE_MIN      = 0.5        # ← 시각화 전용 임계값(권장: 운영값)
DRAW_OUTDIR_NAME    = "viz"      # 출력 서브폴더명
DRAW_ONLY_DIVOT     = True  # ← True면 Divot 클래스만 시각화
# 박스 스타일
DRAW_THICK_BASE     = 2          # 기본 두께(이미지 크기에 비례하여 가중)
DRAW_FONT_SCALE     = 0.8
DRAW_TEXT_THICK     = 1

# ===[Divot 전용 ROC 테이블/그래프]===============================
DRAW_DIVOT_ROC     = True
TARGET_CLASS_NAME  = "Divot"
# 기존: ROC_THRESH_GRID = np.linspace(0.0, 1.0, 11)
# 수정 ↓↓↓
ROC_THRESH_GRID    = np.concatenate(([0.001], np.linspace(0.0, 1.0, 11)))
ROC_OUTDIR_NAME    = "roc_divot"
# ================================================================

# ==============================
# 모델/클래스 로드
# ==============================
model = YOLO(ENGINE)
with open(DATA_YAML, "r") as f:
    data_cfg = yaml.safe_load(f)
CLASS_NAMES = data_cfg["names"]
NUM_CLASSES = len(CLASS_NAMES)

# ==============================
# 예측 (매 실행마다 새 폴더)
# ==============================
project_root = Path("/home/dw/ws_job_msislab/Golf_Project/runs/predict_eval/val_clone")
project_root.mkdir(parents=True, exist_ok=True)
run_name = f"pred_labels_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}"
print(f"🚀 Running TensorRT inference (conf={PRED_CONF}) -> {project_root}/{run_name}")

model.predict(
    source=str(TEST_DIR),
    imgsz=IMG_SIZE,
    conf=PRED_CONF,
    iou=MATCH_IOU,                 # NMS IoU
    device=DEVICE,
    rect=RECT,
    agnostic_nms=AGNOSTIC_NMS,
    augment=AUGMENT,
    deterministic=True,
    seed=1,
    save_txt=True,
    save_conf=True,
    project=project_root,          # 고정 project
    name=run_name,                 # 매 실행 다른 name
    exist_ok=False,                # 동일 이름 있으면 실패 → 새 폴더 강제
    verbose=False,                 # ← per-image 로그 차단
)

PRED_LABEL_DIR = project_root / run_name / "labels"
print(f"✅ Predictions saved to: {PRED_LABEL_DIR}")

# ==============================
# 유틸
# ==============================
def load_yolo_txt(path: Path):
    if (path is None) or (not path.exists()):
        return np.zeros((0, 6), dtype=np.float32)
    rows = []
    with open(path, "r") as f:
        for line in f:
            vals = list(map(float, line.strip().split()))
            if len(vals) == 5:
                c, x, y, w, h = vals; conf = 1.0  # GT
            else:
                c, x, y, w, h, conf = vals       # Pred
            rows.append([c, x, y, w, h, conf])
    return np.asarray(rows, dtype=np.float32)

def xywhn2xyxy_pixels(xywhn, W, H):
    """정규화 xywh → 픽셀 단위 xyxy"""
    if len(xywhn) == 0:
        return np.zeros((0, 4), dtype=np.float32)
    x, y, w, h = xywhn.T
    x1 = (x - w/2) * W
    y1 = (y - h/2) * H
    x2 = (x + w/2) * W
    y2 = (y + h/2) * H
    return np.stack([x1, y1, x2, y2], axis=1).astype(np.float32)

def stable_conf_sort(conf: np.ndarray):
    idx = np.arange(len(conf))
    return np.lexsort((idx, -conf))

def cat_or_empty(arrs, axis=0, empty_shape=None, dtype=None):
    arrs = [a for a in arrs if a is not None]
    if len(arrs) == 0:
        return np.zeros(empty_shape, dtype=dtype)
    try:
        return np.concatenate(arrs, axis=axis)
    except Exception:
        return np.zeros(empty_shape, dtype=dtype)

# ==============================
# 평가 준비 (GT 기준)
# ==============================
gt_files = sorted(GT_LABEL_DIR.glob("*.txt"))
pred_map = {p.stem: p for p in PRED_LABEL_DIR.glob("*.txt")}

# ==============================
# mAP (val@0.001 clone: conf 컷 없음)
# ==============================
tp_list, conf_list, pred_cls_list, target_cls_list = [], [], [], []
for gf in tqdm(gt_files, desc="Collecting stats for mAP (GT-based)..."):
    stem = gf.stem
    pf = pred_map.get(stem, None)
    preds = load_yolo_txt(pf)
    if len(preds):
        preds = preds[preds[:, 5] >= 0.5]   # ✅ conf ≥ 0.5인 박스만 사용 (mAP용)
    gts   = load_yolo_txt(gf)

    gt_boxes   = xywhn2xyxy_pixels(gts[:, 1:5], W=1920, H=1536) if len(gts) else np.zeros((0,4), dtype=np.float32)
    gt_cls     = gts[:, 0].astype(int)  if len(gts) else np.zeros((0,), dtype=int)
    pred_boxes = xywhn2xyxy_pixels(preds[:, 1:5], W=1920, H=1536) if len(preds) else np.zeros((0,4), dtype=np.float32)
    pred_cls   = preds[:, 0].astype(int) if len(preds) else np.zeros((0,), dtype=int)
    conf       = preds[:, 5].astype(np.float32) if len(preds) else np.zeros((0,), dtype=np.float32)

    correct = np.zeros((len(preds), len(IOU_THRS)), dtype=bool)
    if len(preds) and len(gts):
        ious = box_iou(torch.tensor(pred_boxes, dtype=torch.float32),
                       torch.tensor(gt_boxes,   dtype=torch.float32)).numpy()
        order = stable_conf_sort(conf)
        pred_boxes, pred_cls, conf = pred_boxes[order], pred_cls[order], conf[order]
        ious = ious[order]
        for j, thr in enumerate(IOU_THRS):
            detected = []
            for k, pc in enumerate(pred_cls):
                best_iou, best_gt = 0.0, -1
                for gi, gc in enumerate(gt_cls):
                    if gc != pc or gi in detected:
                        continue
                    iou_val = ious[k, gi]
                    if iou_val > best_iou:
                        best_iou, best_gt = iou_val, gi
                if best_iou > thr:   # YOLO val과 동일: >
                    correct[k, j] = True
                    detected.append(best_gt)

    tp_list.append(correct)
    conf_list.append(conf)
    pred_cls_list.append(pred_cls)
    target_cls_list.append(gt_cls)

tp             = cat_or_empty(tp_list, axis=0, empty_shape=(0, len(IOU_THRS)), dtype=bool)
conf_all       = cat_or_empty(conf_list, axis=0, empty_shape=(0,), dtype=np.float32)
pred_cls_all   = cat_or_empty(pred_cls_list, axis=0, empty_shape=(0,), dtype=int)
target_cls_all = cat_or_empty(target_cls_list, axis=0, empty_shape=(0,), dtype=int)

tp_c, fp_c, p, r, f1, ap, ap_class, *_ = ap_per_class(tp, conf_all, pred_cls_all, target_cls_all)

# ---- 안전 캐스팅 & 요약값 계산 (프린트 전에 실행) ----
p_arr  = np.asarray(p,  dtype=float).reshape(-1)
r_arr  = np.asarray(r,  dtype=float).reshape(-1)
ap_np  = np.asarray(ap, dtype=float)

# ap 모양 방어: (n_class, n_iou) 가 아닐 때 정리
if ap_np.ndim == 1:
    n_iou = len(IOU_THRS)
    assert ap_np.size % n_iou == 0, "ap shape unexpected; cannot reshape"
    ap_np = ap_np.reshape(-1, n_iou)

p_mean   = float(p_arr.mean()) if p_arr.size else 0.0
r_mean   = float(r_arr.mean()) if r_arr.size else 0.0
mAP50    = float(ap_np[:, 0].mean()) if ap_np.size else 0.0
mAP5095  = float(ap_np.mean())       if ap_np.size else 0.0

# per-class dict (프린트 루프에서 사용)
ap_dict = {
    int(c): (float(p_arr[i]) if i < p_arr.size else 0.0,
             float(r_arr[i]) if i < r_arr.size else 0.0,
             float(ap_np[i, 0]) if i < ap_np.shape[0] else 0.0,
             float(ap_np[i].mean()) if i < ap_np.shape[0] else 0.0)
    for i, c in enumerate(ap_class)
}

# ==============================
# AUROC + TP/FP/FN/GT  (FN을 점수 0 양성으로 반영)
# ==============================
per_true  = {i: [] for i in range(NUM_CLASSES)}
per_score = {i: [] for i in range(NUM_CLASSES)}
tp_cnt = np.zeros(NUM_CLASSES, dtype=int)
fp_cnt = np.zeros(NUM_CLASSES, dtype=int)
fn_cnt = np.zeros(NUM_CLASSES, dtype=int)
gt_cnt = np.zeros(NUM_CLASSES, dtype=int)

for gf in tqdm(gt_files, desc="Matching for AUROC (GT-based)..."):
    stem = gf.stem
    pf = pred_map.get(stem, None)
    preds = load_yolo_txt(pf)
    gts   = load_yolo_txt(gf)

    if len(gts):
        for c in gts[:, 0].astype(int):
            gt_cnt[c] += 1

    # 예측 없음 + GT 존재 → 전부 FN (점수 0의 양성으로 추가)
    if len(preds) == 0 and len(gts) > 0:
        for c in gts[:, 0].astype(int):
            fn_cnt[c] += 1
            per_true[c].append(1)     # 양성
            per_score[c].append(0.0)  # 미검출 → 점수 0
        continue

    gt_boxes   = xywhn2xyxy_pixels(gts[:, 1:5], W=1920, H=1536) if len(gts) else np.zeros((0,4), dtype=np.float32)
    gt_cls     = gts[:, 0].astype(int)  if len(gts) else np.zeros((0,), dtype=int)
    pred_boxes = xywhn2xyxy_pixels(preds[:, 1:5], W=1920, H=1536) if len(preds) else np.zeros((0,4), dtype=np.float32)
    pred_cls   = preds[:, 0].astype(int) if len(preds) else np.zeros((0,), dtype=int)
    conf       = preds[:, 5].astype(np.float32) if len(preds) else np.zeros((0,), dtype=np.float32)

    if len(preds) and len(gts):
        ious = box_iou(torch.tensor(pred_boxes, dtype=torch.float32),
                       torch.tensor(gt_boxes,   dtype=torch.float32)).numpy()
        order = stable_conf_sort(conf)
        pred_boxes, pred_cls, conf = pred_boxes[order], pred_cls[order], conf[order]
        ious = ious[order]

        used = set()
        for k, (pc, cf) in enumerate(zip(pred_cls, conf)):
            best_iou, best_gt = 0.0, -1
            for gi, gc in enumerate(gt_cls):
                if gc != pc or gi in used:
                    continue
                iou_val = ious[k, gi]
                if iou_val > best_iou:
                    best_iou, best_gt = iou_val, gi
            if best_iou > MATCH_IOU and best_gt != -1:      # '>' 로 통일
                tp_cnt[pc] += 1
                per_true[pc].append(1)
                per_score[pc].append(cf)
                used.add(best_gt)
            else:
                fp_cnt[pc] += 1
                per_true[pc].append(0)
                per_score[pc].append(cf)

        # 매칭되지 않은 GT → FN을 점수 0 양성으로 추가
        for gi, gc in enumerate(gt_cls):
            if gi not in used:
                fn_cnt[gc] += 1
                per_true[gc].append(1)
                per_score[gc].append(0.0)
    elif len(preds) > 0 and len(gts) == 0:
        for c, cf in zip(pred_cls, conf):
            fp_cnt[c] += 1
            per_true[c].append(0)
            per_score[c].append(cf)

aurocs = []
for c in range(NUM_CLASSES):
    y_true  = np.asarray(per_true[c], dtype=np.float32)
    y_score = np.asarray(per_score[c], dtype=np.float32)
    if len(y_true) < 2 or len(np.unique(y_true)) < 2:
        aurocs.append(float("nan"))
    else:
        try:
            aurocs.append(roc_auc_score(y_true, y_score))
        except ValueError:
            aurocs.append(float("nan"))

mean_auroc = np.nanmean(aurocs) if np.any(~np.isnan(aurocs)) else float("nan")

# ==============================================================
# 🔸 (교체) Print용 TP/FP/TN/FN/GT를 conf ≥ EVAL_CONF_PRINT 기준으로 재집계
#     - AUROC 값은 그대로(전체 점수 사용), 숫자 요약만 임계값 기반
#     - per_true/per_score와 동일 공간에서 계산 → ROC와 일관
# ==============================================================
EVAL_CONF_PRINT = 0.5  # 원하는 기준값

tp_thr = np.zeros(NUM_CLASSES, dtype=int)
fp_thr = np.zeros(NUM_CLASSES, dtype=int)
tn_thr = np.zeros(NUM_CLASSES, dtype=int)
fn_thr = np.zeros(NUM_CLASSES, dtype=int)
gt_thr = np.zeros(NUM_CLASSES, dtype=int)  # 양성 수 P

for c in range(NUM_CLASSES):
    y_true  = np.asarray(per_true[c],  dtype=np.int32)    # 1=양성, 0=음성
    y_score = np.asarray(per_score[c], dtype=np.float32)  # 점수(미검출은 0 포함)
    if y_true.size == 0:
        continue

    y_pred = (y_score >= EVAL_CONF_PRINT).astype(np.int32)

    tp_thr[c] = int(((y_pred == 1) & (y_true == 1)).sum())
    fp_thr[c] = int(((y_pred == 1) & (y_true == 0)).sum())
    tn_thr[c] = int(((y_pred == 0) & (y_true == 0)).sum())
    fn_thr[c] = int(((y_pred == 0) & (y_true == 1)).sum())
    gt_thr[c] = int((y_true == 1).sum())

# ==============================
# (NEW) Divot ROC 테이블 & 곡선 (토글)
# ==============================
if DRAW_DIVOT_ROC:
    try:
        CID = CLASS_NAMES.index(TARGET_CLASS_NAME)
    except ValueError:
        CID = None

    if CID is None:
        print(f"[ROC] 클래스 '{TARGET_CLASS_NAME}'를 names에서 찾지 못했습니다. 스킵합니다.")
    else:
        y_true_div  = np.asarray(per_true[CID], dtype=np.int32)
        y_score_div = np.asarray(per_score[CID], dtype=np.float32)

        if y_true_div.size == 0 or len(np.unique(y_true_div)) < 2:
            print(f"[ROC] '{TARGET_CLASS_NAME}'에 유효한 양성/음성이 부족합니다. 스킵합니다.")
        else:
            roc_dir = (project_root / run_name / ROC_OUTDIR_NAME)
            roc_dir.mkdir(parents=True, exist_ok=True)

            # 1) 임의 threshold 그리드로 TP/FP/TN/FN/TPR/FPR 표 계산
            rows = []
            P = int((y_true_div == 1).sum())
            N = int((y_true_div == 0).sum())
            for thr in ROC_THRESH_GRID:
                y_pred = (y_score_div >= thr).astype(np.int32)
                TP = int(((y_pred == 1) & (y_true_div == 1)).sum())
                FP = int(((y_pred == 1) & (y_true_div == 0)).sum())
                TN = int(((y_pred == 0) & (y_true_div == 0)).sum())
                FN = int(((y_pred == 0) & (y_true_div == 1)).sum())
                TPR = (TP / P) if P > 0 else 0.0
                FPR = (FP / N) if N > 0 else 0.0
                rows.append([thr, TP, FP, TN, FN, TPR, FPR])
            rows.sort(key=lambda x: x[0])

            csv_path = roc_dir / "divot_roc_table.csv"
            with open(csv_path, "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(["threshold", "TP", "FP", "TN", "FN", "TPR", "FPR"])
                writer.writerows(rows)
            print(f"[ROC] Divot ROC table saved: {csv_path}")

            # 콘솔에도 간단 프리뷰
            print("\n[ROC] Divot — threshold sweep preview")
            print("thr     TP    FP    TN    FN    TPR      FPR")
            for r in rows[:11]:
                print(f"{r[0]:.3f}  {r[1]:5d} {r[2]:5d} {r[3]:5d} {r[4]:5d}  {r[5]:.4f}  {r[6]:.4f}")

            # 2) sklearn으로 ROC 곡선 본판 계산/그리기 (모든 점수 유니크 임계값)
            fpr, tpr, thr = roc_curve(y_true_div, y_score_div)
            roc_auc = auc(fpr, tpr)

            plt.figure(figsize=(5,5))
            plt.plot(fpr, tpr, marker='.', linewidth=1, label=f'ROC (AUC={roc_auc:.4f})')
            plt.plot([0,1],[0,1],'--', linewidth=1, label='chance')
            plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title(f'ROC — {TARGET_CLASS_NAME}')
            plt.legend()
            plt.grid(True, alpha=0.3)
            fig_path = roc_dir / "divot_roc_curve.png"
            plt.savefig(fig_path, bbox_inches="tight", dpi=150)
            plt.close()
            print(f"[ROC] Divot ROC curve saved: {fig_path}")

# ==============================
# 출력 (mAP/AUROC)
# ==============================
print(f"\n================= ✅ MANUAL EVALUATION (mAP@conf≥0.5, AUROC full range) =================")
print(f"{'all':<15}  mP={p_mean:.3f}  mR={r_mean:.3f}  mAP50={mAP50:.3f}  mAP50-95={mAP5095:.3f}")
for i in range(NUM_CLASSES):
    if i in ap_dict:
        pi, ri, ap50_i, ap5095_i = ap_dict[i]
    else:
        pi, ri, ap50_i, ap5095_i = 0.0, 0.0, 0.0, 0.0
    print(f"{CLASS_NAMES[i]:<15}  P={pi:.3f}  R={ri:.3f}  mAP50={ap50_i:.3f}  mAP50-95={ap5095_i:.3f}")
print("======================================================================")

print(f"\n📈 AUROC Evaluation per class (TP/FP/TN/FN/GT @ conf ≥ {EVAL_CONF_PRINT})")
print("─────────────────────────────────────────────────────────────")
for i in range(NUM_CLASSES):
    auc_str = f"{aurocs[i]:.4f}" if not math.isnan(aurocs[i]) else "NaN"
    print(f"{CLASS_NAMES[i]:<18s}: AUROC={auc_str:<8s} | "
          f"TP={tp_thr[i]:<4d} FP={fp_thr[i]:<4d} TN={tn_thr[i]:<4d} "
          f"FN={fn_thr[i]:<4d} GT={gt_thr[i]:<4d}")
print("─────────────────────────────────────────────────────────────")
print(f"🔹 Mean AUROC (macro): {mean_auroc:.4f}" if not math.isnan(mean_auroc) else "🔹 Mean AUROC: NaN")
print("======================================================================")

# ==============================
# (옵션) 시각화/저장 (GT=초록, TP=파랑, FP=빨강, Divot 전용 모드 추가)
# ==============================

def iou(box1, box2):
    """YOLO 형식 (cx, cy, w, h) → IoU 계산"""
    x1_min, y1_min = box1[0]-box1[2]/2, box1[1]-box1[3]/2
    x1_max, y1_max = box1[0]+box1[2]/2, box1[1]+box1[3]/2
    x2_min, y2_min = box2[0]-box2[2]/2, box2[1]-box2[3]/2
    x2_max, y2_max = box2[0]+box2[2]/2, box2[1]+box2[3]/2
    inter_w = max(0, min(x1_max, x2_max) - max(x1_min, x2_min))
    inter_h = max(0, min(y1_max, y2_max) - max(y1_min, y2_min))
    inter_area = inter_w * inter_h
    union_area = box1[2]*box1[3] + box2[2]*box2[3] - inter_area
    return inter_area / union_area if union_area > 0 else 0


if DRAW_SAVE:
    viz_dir = project_root / run_name / DRAW_OUTDIR_NAME
    viz_dir.mkdir(parents=True, exist_ok=True)

    # 클래스 인덱스 필터링
    if DRAW_ONLY_DIVOT:
        try:
            TARGET_CID = CLASS_NAMES.index("Divot")
            print(f"🟦 Divot 전용 시각화 모드 활성화 (class id={TARGET_CID})")
        except ValueError:
            print("⚠️ 'Divot' 클래스를 찾을 수 없습니다. 모든 클래스를 시각화합니다.")
            TARGET_CID = None
    else:
        TARGET_CID = None

    print(f"\n🖼  Drawing GT=green, TP=blue, FP=red to: {viz_dir}")

    for img_path in tqdm(sorted(TEST_DIR.glob("*.jpg")), desc="Drawing"):
        name = img_path.stem
        gt_path = GT_LABEL_DIR / f"{name}.txt"
        pred_path = PRED_LABEL_DIR / f"{name}.txt"
        if not img_path.exists():
            continue

        img = cv2.imread(str(img_path))
        if img is None:
            continue
        H, W = img.shape[:2]

        # GT 로드
        gt_boxes = []
        if gt_path.exists():
            with open(gt_path, "r") as f:
                for line in f:
                    vals = list(map(float, line.strip().split()))
                    if len(vals) < 5:
                        continue
                    cls, x, y, w, h = vals[:5]
                    cls = int(cls)
                    if (TARGET_CID is not None) and (cls != TARGET_CID):
                        continue
                    gt_boxes.append((cls, [x, y, w, h]))

        # 예측 로드
        preds = []
        if pred_path.exists():
            with open(pred_path, "r") as f:
                for line in f:
                    vals = list(map(float, line.strip().split()))
                    if len(vals) < 6:
                        continue
                    cls, x, y, w, h, conf = vals[:6]
                    cls = int(cls)
                    if conf < DRAW_SCORE_MIN:
                        continue
                    if (TARGET_CID is not None) and (cls != TARGET_CID):
                        continue
                    preds.append((cls, [x, y, w, h], conf))

        # --- GT 박스 (초록색) ---
        for cls, box in gt_boxes:
            cx, cy, bw, bh = box
            x1, y1, x2, y2 = int((cx-bw/2)*W), int((cy-bh/2)*H), int((cx+bw/2)*W), int((cy+bh/2)*H)
            color = (0, 255, 0)
            label = f"GT-{CLASS_NAMES[cls] if cls < len(CLASS_NAMES) else cls}"
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, label, (x1, max(y1-5, 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        # --- 예측 박스 (TP=파랑, FP=빨강) ---
        for cls, box, conf in preds:
            cx, cy, bw, bh = box
            x1, y1, x2, y2 = int((cx-bw/2)*W), int((cy-bh/2)*H), int((cx+bw/2)*W), int((cy+bh/2)*H)
            matched = any((cls == g_cls and iou(box, g_box) >= MATCH_IOU) for g_cls, g_box in gt_boxes)
            if matched:
                color = (255, 0, 0)   # TP 파랑
                label = f"TP-{CLASS_NAMES[cls] if cls < len(CLASS_NAMES) else cls} {conf:.2f}"
            else:
                color = (0, 0, 255)   # FP 빨강
                label = f"FP-{CLASS_NAMES[cls] if cls < len(CLASS_NAMES) else cls} {conf:.2f}"

            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, label, (x1, max(y1-5, 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        out_path = viz_dir / f"{name}.jpg"
        cv2.imwrite(str(out_path), img)

    print(f"✅ 전체 클래스 시각화 완료! 결과 폴더: {viz_dir}")

print("\n✅ Done.")


🚀 Running TensorRT inference (conf=0.001) -> /home/dw/ws_job_msislab/Golf_Project/runs/predict_eval/val_clone/pred_labels_20251125_151554_641554
[11/25/2025-15:15:54] [TRT] [W] WARNING The logger passed into createInferRuntime differs from one already registered for an existing builder, runtime, or refitter. So the current new logger is ignored, and TensorRT will use the existing one which is returned by nvinfer1::getLogger() instead.
[11/25/2025-15:15:54] [TRT] [I] Loaded engine size: 25 MiB
[11/25/2025-15:15:54] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +1, GPU +18, now: CPU 1, GPU 77 (MiB)
✅ Predictions saved to: /home/dw/ws_job_msislab/Golf_Project/runs/predict_eval/val_clone/pred_labels_20251125_151554_641554/labels


Matching for AUROC (GT-based)...: 100%|██████████| 1237/1237 [00:00<00:00, 7177.71it/s]


[ROC] Divot ROC table saved: /home/dw/ws_job_msislab/Golf_Project/runs/predict_eval/val_clone/pred_labels_20251125_151554_641554/roc_divot/divot_roc_table.csv

[ROC] Divot — threshold sweep preview
thr     TP    FP    TN    FN    TPR      FPR
0.000   1416   238     0     0  1.0000  1.0000
0.001   1208   238     0   208  0.8531  1.0000
0.100   1122    97   141   294  0.7924  0.4076
0.200   1098    86   152   318  0.7754  0.3613
0.300   1081    78   160   335  0.7634  0.3277
0.400   1064    72   166   352  0.7514  0.3025
0.500   1048    65   173   368  0.7401  0.2731
0.600   1034    57   181   382  0.7302  0.2395
0.700   1012    53   185   404  0.7147  0.2227
0.800    974    35   203   442  0.6879  0.1471
0.900    370     2   236  1046  0.2613  0.0084
[ROC] Divot ROC curve saved: /home/dw/ws_job_msislab/Golf_Project/runs/predict_eval/val_clone/pred_labels_20251125_151554_641554/roc_divot/divot_roc_curve.png

================= ✅ MANUAL EVALUATION (mAP@conf≥0.5, AUROC full range) =========

Drawing: 100%|██████████| 2426/2426 [01:14<00:00, 32.59it/s]

✅ 전체 클래스 시각화 완료! 결과 폴더: /home/dw/ws_job_msislab/Golf_Project/runs/predict_eval/val_clone/pred_labels_20251125_151554_641554/viz

✅ Done.


### 3 모델 결과 (평균치로 계산)

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# ============================================================
# YOLOv8 TensorRT — Multi-Seed (No Groups)
# Parallel Predict → Global Ensemble → mAP (val@0.001 clone) & AUROC
# - 실행마다 고유 폴더 생성
# - per-image 로그 비표시
# - FN(미검출)을 AUROC에 점수 0 양성으로 반영
# - mAP clone과 동일하게 IoU 비교는 '>' 사용
# - (옵션) 앙상블 결과를 원본 이미지에 시각화/저장 (전 클래스)
# - (추가) Divot 전용 ROC 표/그래프 저장
# ============================================================

# ── [무음 설정] ultralytics 임포트 전에 ──
import os, logging, math, gc, random, csv
os.environ["ULTRALYTICS_QUIET"] = "1"   # Ultralytics 내부 로그 억제
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True,max_split_size_mb:128")

from concurrent.futures import ThreadPoolExecutor, as_completed
from ultralytics import YOLO
from ultralytics.utils import LOGGER
from ultralytics.utils.metrics import ap_per_class as _ap_per_class
from pathlib import Path
from tqdm import tqdm
import numpy as np
import torch
import yaml
from datetime import datetime
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import cv2  # ← 시각화용

# ── Ultralytics 로거 레벨 하향 ──
LOGGER.setLevel(logging.ERROR)
torch.set_num_threads(1)

# ==============================
# 공통 설정
# ==============================
TEST_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251112/images/test")
GT_LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251112/labels/test")
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251112/data.yaml"

# ---- 실행마다 고유 폴더 생성 ----
BASE_ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/runs/predict_eval/multimodel_ensemble_eval_no_groups")
RUN_NAME = f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
BASE_DIR = BASE_ROOT / RUN_NAME
BASE_DIR.mkdir(parents=True, exist_ok=True)

# 입력 해상도(정규화 복원용) — (주의) 시각화는 실제 이미지 W,H를 사용함
IMG_W, IMG_H = 1920, 1536

# Predict 설정
IMG_SIZE       = 640
DEVICE         = 0
PRED_CONF      = 0.001     # 저장용 conf (mAP clone은 conf 컷 없음)
PRED_IOU_NMS   = 0.5
AUGMENT        = False
AGNOSTIC_NMS   = True
RECT           = False
MAX_WORKERS    = 4

# Ensemble 설정
IOU_MERGE          = 0.7   # IoU ≥ IOU_MERGE → 동일 객체 간주
CONF_THRESH_SINGLE = 0.5  # 단일 매칭 채택 하한
MIN_MATCH          = 2     # 매칭된 시드 수

# Evaluation 설정
IOU_THRS = np.linspace(0.5, 0.95, 10, dtype=np.float32)  # mAP@0.5:0.95
MATCH_IOU = 0.5   # AUROC/TPFPFN 매칭 임계치 (mAP clone과 동일하게 '>')

# ===[시각화 토글/옵션 (앙상블)]====================================
VIZ_SAVE           = False         # ← True면 앙상블 결과를 원본 이미지에 그려 저장
VIZ_USE_PRED_CONF  = False           # ← True면 시각화 임계값을 PRED_CONF와 동일하게 사용
VIZ_SCORE_MIN      = 0.001           # ← VIZ_USE_PRED_CONF=False일 때 쓰는 별도 임계값
VIZ_OUTDIR_NAME    = "viz"          # 출력 폴더명 (all_ensembles/ 아래에 생성)
VIZ_DIVOT_ONLY     = True      # ✅ True면 Divot만 시각화, False면 전 클래스 표시
# 스타일
VIZ_THICK_BASE     = 2
VIZ_FONT_SCALE     = 0.5
VIZ_TEXT_THICK     = 1

# ===[Divot 전용 ROC 테이블/그래프]===============================
DRAW_DIVOT_ROC     = True
TARGET_CLASS_NAME  = "Divot"
# 기존: ROC_THRESH_GRID = np.linspace(0.0, 1.0, 11)
# 수정 ↓↓↓
ROC_THRESH_GRID    = np.concatenate(([0.001], np.linspace(0.0, 1.0, 11)))
ROC_OUTDIR_NAME    = "roc_divot"
# ================================================================

# ==============================
# 시드별 엔진 (그룹 없음)
# ==============================
ENGINES = {
    "best1": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/best_fp16_bs1_640px_static.engine",
    "best2": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/precision_BEST_epoch70_0.95995_fp16_bs1_640px_static.engine",
    "best3": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/best_fp16_bs1_640px_static.engine",
}
# ==============================
# 유틸
# ==============================
def load_yolo_txt(path: Path):
    if (path is None) or (not path.exists()):
        return np.zeros((0, 6), dtype=np.float32)
    rows = []
    with open(path, "r") as f:
        for line in f:
            vals = list(map(float, line.strip().split()))
            if len(vals) == 5:  # GT
                c, x, y, w, h = vals; conf = 1.0
            else:               # Pred
                c, x, y, w, h, conf = vals
            rows.append([c, x, y, w, h, conf])
    return np.asarray(rows, dtype=np.float32)

def xywh2xyxy(boxes, W=IMG_W, H=IMG_H):
    if len(boxes) == 0: return np.zeros((0, 4), dtype=np.float32)
    x, y, w, h = boxes.T
    x1 = (x - w/2) * W; y1 = (y - h/2) * H
    x2 = (x + w/2) * W; y2 = (y + h/2) * H
    return np.stack([x1, y1, x2, y2], axis=1).astype(np.float32)

def xyxy2xywh(boxes, W=IMG_W, H=IMG_H):
    if len(boxes) == 0: return np.zeros((0, 4), dtype=np.float32)
    x1, y1, x2, y2 = boxes.T
    x = (x1 + x2) / 2 / W; y = (y1 + y2) / 2 / H
    w = (x2 - x1) / W;     h = (y2 - y1) / H
    return np.stack([x, y, w, h], axis=1).astype(np.float32)

def box_iou_np(a, b):
    if len(a) == 0 or len(b) == 0:
        return np.zeros((len(a), len(b)), dtype=np.float32)
    area1 = (a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1])
    area2 = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
    ix1 = np.maximum(a[:, None, 0], b[:, 0]); iy1 = np.maximum(a[:, None, 1], b[:, 1])
    ix2 = np.minimum(a[:, None, 2], b[:, 2]); iy2 = np.minimum(a[:, None, 3], b[:, 3])
    iw = np.clip(ix2 - ix1, 0, None); ih = np.clip(iy2 - iy1, 0, None)
    inter = iw * ih
    return inter / (area1[:, None] + area2 - inter + 1e-6)

def stable_conf_sort(conf: np.ndarray):
    idx = np.arange(len(conf))
    return np.lexsort((idx, -conf))

def cat_or_empty(arrs, axis=0, empty_shape=None, dtype=None):
    arrs = [a for a in arrs if a is not None]
    if len(arrs) == 0:
        return np.zeros(empty_shape, dtype=dtype)
    try:
        return np.concatenate(arrs, axis=axis)
    except Exception:
        return np.zeros(empty_shape, dtype=dtype)

def ap_per_class_compat(tp, conf, pred_cls, target_cls):
    out = _ap_per_class(tp, conf, pred_cls, target_cls)
    if len(out) == 5:
        p, r, ap, f1, ap_class = out
    elif len(out) >= 7:
        p, r, ap, f1, ap_class = out[2], out[3], out[5], out[4], out[6]
    else:
        raise RuntimeError("Unexpected ap_per_class return format")
    return p, r, ap, f1, ap_class

def xywhn2xyxy_pixels(xywhn, W, H):
    if len(xywhn) == 0:
        return np.zeros((0, 4), dtype=np.float32)
    x, y, w, h = xywhn.T
    x1 = (x - w/2) * W
    y1 = (y - h/2) * H
    x2 = (x + w/2) * W
    y2 = (y + h/2) * H
    return np.stack([x1, y1, x2, y2], axis=1).astype(np.float32)

def _class_colors(n):
    colors = []
    for i in range(n):
        random.seed(i + 12345)
        colors.append((random.randint(60,255), random.randint(60,255), random.randint(60,255)))
    return colors

# ==============================
# 클래스 로드
# ==============================
with open(DATA_YAML, "r") as f:
    data_cfg = yaml.safe_load(f)
CLASS_NAMES = data_cfg["names"]
NUM_CLASSES = len(CLASS_NAMES)

# ==============================
# 1) Predict (스레드 병렬, 기본=순차)
# ==============================
print(f"🚀 Parallel inference for seeds (no groups) ... → {BASE_DIR}")
predict_jobs = []  # (seed, out_label_dir)

def _predict_one(seed_name, engine_path):
    out_dir = BASE_DIR / "raw_preds" / seed_name
    out_dir.mkdir(parents=True, exist_ok=True)
    model = YOLO(engine_path)
    model.predict(
        source=str(TEST_DIR),
        imgsz=IMG_SIZE,
        conf=PRED_CONF,
        iou=PRED_IOU_NMS,
        device=DEVICE,
        rect=RECT,
        agnostic_nms=AGNOSTIC_NMS,
        augment=AUGMENT,
        deterministic=True,
        seed=1,
        save_txt=True,
        save_conf=True,
        project=out_dir,
        name="pred_labels",
        exist_ok=True,
        verbose=False,
    )
    try:
        model.predictor = None
    except Exception:
        pass
    del model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.synchronize(); torch.cuda.empty_cache()
    return out_dir / "pred_labels" / "labels"

futures = []
with ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(ENGINES))) as ex:
    for sname, eng in ENGINES.items():
        futures.append(ex.submit(_predict_one, sname, eng))
    for fut in tqdm(as_completed(futures), total=len(futures), desc="Predicting"):
        try:
            label_dir = fut.result()
            predict_jobs.append((Path(label_dir).name, label_dir))
            print(f"✅ {label_dir}")
        except Exception as e:
            print("❌ Predict failed:", e)

print(f"🎯 총 {len(predict_jobs)}개 시드 예측 완료")
if not predict_jobs:
    raise SystemExit("No predictions were produced. Check engines/paths.")

# ==============================
# 2) 글로벌 앙상블 (모든 시드 합침)
# ==============================
def ensemble_merge(label_dirs, out_dir: Path,
                   iou_thr=IOU_MERGE, min_match=MIN_MATCH, conf_single=CONF_THRESH_SINGLE):
    out_dir.mkdir(parents=True, exist_ok=True)
    base = label_dirs[0]
    all_files = sorted(os.listdir(base))

    for file in tqdm(all_files, desc=f"Ensembling → {out_dir.name}"):
        preds_all = [load_yolo_txt(Path(d) / file) for d in label_dirs]
        combined = np.concatenate(preds_all, axis=0) if len(preds_all) else np.zeros((0, 6), dtype=np.float32)
        if len(combined) == 0:
            open(out_dir / file, "w").close()
            continue

        boxes_xyxy = xywh2xyxy(combined[:, 1:5])
        cls = combined[:, 0].astype(int)
        conf = combined[:, 5].astype(np.float32)

        used = np.zeros(len(combined), dtype=bool)
        final_preds = []

        for i in range(len(combined)):
            if used[i]:
                continue
            same_cls_idx = np.where(cls == cls[i])[0]
            ious = box_iou_np(boxes_xyxy[i:i+1], boxes_xyxy[same_cls_idx])[0]
            matched = same_cls_idx[ious >= iou_thr]
            used[matched] = True

            k = len(matched)
            if k >= min_match:
                avg_box = boxes_xyxy[matched].mean(axis=0)
                avg_conf = conf[matched].mean()
                final_preds.append([cls[i], *avg_box, avg_conf])
            elif k == 1 and conf[i] >= conf_single:
                final_preds.append([cls[i], *boxes_xyxy[i], conf[i]])

        if len(final_preds):
            final_preds = np.array(final_preds, dtype=np.float32)
            xywh = xyxy2xywh(final_preds[:, 1:5])
            out = np.column_stack([final_preds[:, 0], xywh, final_preds[:, 5]])
            np.savetxt(out_dir / file, out, fmt="%.6f")
        else:
            open(out_dir / file, "w").close()

ALL_LABEL_DIRS = [ld for _, ld in predict_jobs]
ALL_ENSEMBLE_DIR = BASE_DIR / "all_ensembles" / "labels"
ensemble_merge(ALL_LABEL_DIRS, ALL_ENSEMBLE_DIR)
print(f"✅ Global ensemble saved: {ALL_ENSEMBLE_DIR}")

# ==============================
# 3) mAP clone(val@0.001) + AUROC + Divot ROC
# ==============================
def evaluate_map_auroc(PRED_LABEL_DIR: Path, title: str):
    print(f"\n================= ✅ EVALUATION — {title} =================")
    gt_files = sorted(GT_LABEL_DIR.glob("*.txt"))
    pred_map = {p.stem: p for p in PRED_LABEL_DIR.glob("*.txt")}

    # ----- mAP clone -----
    tp_list, conf_list, pred_cls_list, target_cls_list = [], [], [], []
    for gf in tqdm(gt_files, desc="Collecting stats for mAP..."):
        stem = gf.stem
        pf = pred_map.get(stem, None)

        preds = load_yolo_txt(pf)
        gts   = load_yolo_txt(gf)

        gt_boxes   = xywh2xyxy(gts[:, 1:5]) if len(gts) else np.zeros((0, 4), dtype=np.float32)
        gt_cls     = gts[:, 0].astype(int)  if len(gts) else np.zeros((0,), dtype=int)
        pred_boxes = xywh2xyxy(preds[:, 1:5]) if len(preds) else np.zeros((0, 4), dtype=np.float32)
        pred_cls   = preds[:, 0].astype(int) if len(preds) else np.zeros((0,), dtype=int)
        conf       = preds[:, 5].astype(np.float32) if len(preds) else np.zeros((0,), dtype=np.float32)

        correct = np.zeros((len(preds), len(IOU_THRS)), dtype=bool)
        if len(preds) and len(gts):
            ious = box_iou_np(pred_boxes, gt_boxes)
            order = stable_conf_sort(conf)
            pred_boxes, pred_cls, conf = pred_boxes[order], pred_cls[order], conf[order]
            ious = ious[order]
            for j, thr in enumerate(IOU_THRS):
                detected = []
                for k, pc in enumerate(pred_cls):
                    best_iou, best_gt = 0.0, -1
                    for gi, gc in enumerate(gt_cls):
                        if gc != pc or gi in detected:
                            continue
                        iou_val = ious[k, gi]
                        if iou_val > best_iou:
                            best_iou, best_gt = iou_val, gi
                    if best_iou > thr:   # val과 동일: >
                        correct[k, j] = True
                        detected.append(best_gt)

        tp_list.append(correct)
        conf_list.append(conf)
        pred_cls_list.append(pred_cls)
        target_cls_list.append(gt_cls)

    tp             = cat_or_empty(tp_list, axis=0, empty_shape=(0, len(IOU_THRS)), dtype=bool)
    conf_all       = cat_or_empty(conf_list, axis=0, empty_shape=(0,), dtype=np.float32)
    pred_cls_all   = cat_or_empty(pred_cls_list, axis=0, empty_shape=(0,), dtype=int)
    target_cls_all = cat_or_empty(target_cls_list, axis=0, empty_shape=(0,), dtype=int)

    p, r, ap, f1, ap_class = ap_per_class_compat(tp, conf_all, pred_cls_all, target_cls_all)

    # 안전 캐스팅
    p_arr = np.asarray(p, dtype=float).reshape(-1)
    r_arr = np.asarray(r, dtype=float).reshape(-1)
    ap_np = np.asarray(ap, dtype=float)
    if ap_np.ndim == 1:
        n_iou = len(IOU_THRS)
        assert ap_np.size % n_iou == 0, "ap shape unexpected; cannot reshape"
        ap_np = ap_np.reshape(-1, n_iou)

    mAP50   = float(ap_np[:, 0].mean()) if ap_np.size else 0.0
    mAP5095 = float(ap_np.mean()) if ap_np.size else 0.0
    ap_dict = {
        int(c): (float(p_arr[i]) if i < p_arr.size else 0.0,
                 float(r_arr[i]) if i < r_arr.size else 0.0,
                 float(ap_np[i, 0]) if i < ap_np.shape[0] else 0.0,
                 float(ap_np[i].mean()) if i < ap_np.shape[0] else 0.0)
        for i, c in enumerate(ap_class)
    }

    print(f"{'all':<15}  mP={p_arr.mean():.3f}  mR={r_arr.mean():.3f}  mAP50={mAP50:.3f}  mAP50-95={mAP5095:.3f}")
    for i in range(NUM_CLASSES):
        if i in ap_dict:
            pi, ri, ap50_i, ap5095_i = ap_dict[i]
        else:
            pi, ri, ap50_i, ap5095_i = 0.0, 0.0, 0.0, 0.0
        print(f"{CLASS_NAMES[i]:<15}  P={pi:.3f}  R={ri:.3f}  mAP50={ap50_i:.3f}  mAP50-95={ap5095_i:.3f}")

    # ----- AUROC (FN을 점수 0으로 반영) -----
    per_true  = {i: [] for i in range(NUM_CLASSES)}
    per_score = {i: [] for i in range(NUM_CLASSES)}
    tp_cnt = np.zeros(NUM_CLASSES, dtype=int)
    fp_cnt = np.zeros(NUM_CLASSES, dtype=int)
    fn_cnt = np.zeros(NUM_CLASSES, dtype=int)
    gt_cnt = np.zeros(NUM_CLASSES, dtype=int)

    for gf in tqdm(gt_files, desc="Matching for AUROC..."):
        stem = gf.stem
        pf = pred_map.get(stem, None)

        preds = load_yolo_txt(pf)
        gts   = load_yolo_txt(gf)

        if len(gts):
            for c in gts[:, 0].astype(int):
                gt_cnt[c] += 1

        if len(preds) == 0 and len(gts) > 0:
            for c in gts[:, 0].astype(int):
                fn_cnt[c] += 1
                per_true[c].append(1)
                per_score[c].append(0.0)
            continue

        gt_boxes   = xywh2xyxy(gts[:, 1:5]) if len(gts) else np.zeros((0, 4), dtype=np.float32)
        gt_cls     = gts[:, 0].astype(int)  if len(gts) else np.zeros((0,), dtype=int)
        pred_boxes = xywh2xyxy(preds[:, 1:5]) if len(preds) else np.zeros((0, 4), dtype=np.float32)
        pred_cls   = preds[:, 0].astype(int) if len(preds) else np.zeros((0,), dtype=int)
        conf       = preds[:, 5].astype(np.float32) if len(preds) else np.zeros((0,), dtype=np.float32)

        if len(preds) and len(gts):
            ious = box_iou_np(pred_boxes, gt_boxes)
            order = stable_conf_sort(conf)
            pred_boxes, pred_cls, conf = pred_boxes[order], pred_cls[order], conf[order]
            ious = ious[order]

            used = set()
            for k, (pc, cf) in enumerate(zip(pred_cls, conf)):
                best_iou, best_gt = 0.0, -1
                for gi, gc in enumerate(gt_cls):
                    if gc != pc or gi in used:
                        continue
                    iou_val = ious[k, gi]
                    if iou_val > best_iou:
                        best_iou, best_gt = iou_val, gi
                if best_iou > MATCH_IOU and best_gt != -1:
                    tp_cnt[pc] += 1
                    per_true[pc].append(1)
                    per_score[pc].append(cf)
                    used.add(best_gt)
                else:
                    fp_cnt[pc] += 1
                    per_true[pc].append(0)
                    per_score[pc].append(cf)

            for gi, gc in enumerate(gt_cls):
                if gi not in used:
                    fn_cnt[gc] += 1
                    per_true[gc].append(1)
                    per_score[gc].append(0.0)
        elif len(preds) > 0 and len(gts) == 0:
            for c, cf in zip(pred_cls, conf):
                fp_cnt[c] += 1
                per_true[c].append(0)
                per_score[c].append(cf)

    # ── Divot 전용 ROC 저장 ──
    if DRAW_DIVOT_ROC and TARGET_CLASS_NAME in CLASS_NAMES:
        divot_id = CLASS_NAMES.index(TARGET_CLASS_NAME)
        y_true  = np.asarray(per_true[divot_id], dtype=np.float32)
        y_score = np.asarray(per_score[divot_id], dtype=np.float32)

        roc_dir = BASE_DIR / ROC_OUTDIR_NAME
        roc_dir.mkdir(parents=True, exist_ok=True)

        # ROC 곡선
        if len(y_true) >= 2 and len(np.unique(y_true)) >= 2:
            fpr, tpr, _ = roc_curve(y_true, y_score)
            roc_auc = auc(fpr, tpr)
        else:
            fpr, tpr, roc_auc = np.array([0,1]), np.array([0,1]), float("nan")

        plt.figure(figsize=(5,5))
        plt.plot(fpr, tpr, marker='.', linewidth=1, label=f'ROC (AUC={roc_auc:.4f})')
        plt.plot([0,1],[0,1],'--', linewidth=1, label='chance')
        plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title(f'ROC — {TARGET_CLASS_NAME}')
        plt.legend(); plt.grid(True, alpha=0.3)
        fig_path = roc_dir / "divot_roc_curve.png"
        plt.savefig(fig_path, bbox_inches="tight", dpi=150)
        plt.close()
        print(f"[ROC] Divot ROC curve saved: {fig_path}")

        # 임계값 스윕 표
        rows = []
        P = int((y_true == 1).sum())
        N = int((y_true == 0).sum())
        for thr in ROC_THRESH_GRID:
            y_pred = (y_score >= thr).astype(np.int32)
            TP = int(((y_pred == 1) & (y_true == 1)).sum())
            FP = int(((y_pred == 1) & (y_true == 0)).sum())
            TN = int(((y_pred == 0) & (y_true == 0)).sum())
            FN = int(((y_pred == 0) & (y_true == 1)).sum())
            TPR = (TP / P) if P > 0 else 0.0
            FPR = (FP / N) if N > 0 else 0.0
            rows.append((thr, TP, FP, TN, FN, TPR, FPR))
        rows.sort(key=lambda x: x[0])

        csv_path = roc_dir / "divot_roc_table.csv"
        with open(csv_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["threshold", "TP", "FP", "TN", "FN", "TPR", "FPR"])
            writer.writerows(rows)
        print(f"[ROC] Divot ROC table saved: {csv_path}")

        # 콘솔 프리뷰
        print("\n[ROC] Divot — threshold sweep preview")
        print("thr     TP    FP    TN    FN    TPR      FPR")
        for r in rows[:11]:
            print(f"{r[0]:.3f}  {r[1]:5d} {r[2]:5d} {r[3]:5d} {r[4]:5d}  {r[5]:.4f}  {r[6]:.4f}")

    # ── AUROC 표준 출력 ──
    print("\n📈 AUROC Evaluation per class")
    print("─────────────────────────────────────────────────────────────")
    aurocs = []
    for i in range(NUM_CLASSES):
        y_true_i  = np.asarray(per_true[i], dtype=np.float32)
        y_score_i = np.asarray(per_score[i], dtype=np.float32)
        if len(y_true_i) < 2 or len(np.unique(y_true_i)) < 2:
            auroc_i = float("nan")
        else:
            try:
                auroc_i = roc_auc_score(y_true_i, y_score_i)
            except ValueError:
                auroc_i = float("nan")
        aurocs.append(auroc_i)
        auc_str = f"{auroc_i:.4f}" if not math.isnan(auroc_i) else "NaN"
        print(f"{CLASS_NAMES[i]:<18s}: AUROC={auc_str:<8s} | "
              f"TP={tp_cnt[i]:<4d} FP={fp_cnt[i]:<4d} FN={fn_cnt[i]:<4d} GT={gt_cnt[i]:<4d}")
    print("─────────────────────────────────────────────────────────────")
    mean_auroc = np.nanmean(aurocs) if np.any(~np.isnan(aurocs)) else float("nan")
    if not math.isnan(mean_auroc):
        print(f"🔹 Mean AUROC (macro): {mean_auroc:.4f}")
    else:
        print("🔹 Mean AUROC: NaN")
    print("======================================================================")

# 실행 (글로벌만)
evaluate_map_auroc(ALL_ENSEMBLE_DIR, title=f"Global Ensemble — ALL SEEDS ({RUN_NAME})")

# ==============================
# 4) (옵션) 앙상블 시각화/저장 — GT/TP/FP 구분 + Divot 전용 모드
# ==============================
if VIZ_SAVE:
    viz_dir = BASE_DIR / "all_ensembles" / VIZ_OUTDIR_NAME
    viz_dir.mkdir(parents=True, exist_ok=True)
    thr = PRED_CONF if VIZ_USE_PRED_CONF else VIZ_SCORE_MIN

    print(f"\n🖼  Drawing ENSEMBLE predictions (GT=green, TP=blue, FP=red)"
          f"{' [DIVOT ONLY]' if VIZ_DIVOT_ONLY else ''} → {viz_dir} (thr={thr})")

    img_paths = [p for p in sorted(TEST_DIR.glob("*"))
                 if p.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}]

    def iou(box1, box2):
        x1_min, y1_min = box1[0]-box1[2]/2, box1[1]-box1[3]/2
        x1_max, y1_max = box1[0]+box1[2]/2, box1[1]+box1[3]/2
        x2_min, y2_min = box2[0]-box2[2]/2, box2[1]-box2[3]/2
        x2_max, y2_max = box2[0]+box2[2]/2, box2[1]+box2[3]/2
        inter_w = max(0, min(x1_max, x2_max) - max(x1_min, x2_min))
        inter_h = max(0, min(y1_max, y2_max) - max(y1_min, y2_min))
        inter_area = inter_w * inter_h
        union_area = box1[2]*box1[3] + box2[2]*box2[3] - inter_area
        return inter_area / union_area if union_area > 0 else 0.0

    # Divot 클래스 ID 찾기
    divot_id = None
    if "Divot" in CLASS_NAMES:
        divot_id = CLASS_NAMES.index("Divot")

    for img_path in tqdm(img_paths, desc="Drawing (GT/TP/FP)"):
        stem = img_path.stem
        gt_path = GT_LABEL_DIR / f"{stem}.txt"
        pred_path = ALL_ENSEMBLE_DIR / f"{stem}.txt"
        if not gt_path.exists() and not pred_path.exists():
            continue

        img = cv2.imread(str(img_path))
        if img is None:
            continue
        H, W = img.shape[:2]
        thick = max(1, int(VIZ_THICK_BASE * (W + H) / 2000))

        # ── GT 로드 ──
        gt_boxes = []
        if gt_path.exists():
            with open(gt_path, "r") as f:
                for line in f:
                    vals = list(map(float, line.strip().split()))
                    if len(vals) >= 5:
                        cls = int(vals[0])
                        if VIZ_DIVOT_ONLY and cls != divot_id:
                            continue
                        gt_boxes.append((cls, vals[1:5]))

        # ── 예측 로드 ──
        preds = []
        if pred_path.exists():
            with open(pred_path, "r") as f:
                for line in f:
                    vals = list(map(float, line.strip().split()))
                    if len(vals) >= 6:
                        cls, x, y, w, h, conf = vals
                        if conf < thr:
                            continue
                        if VIZ_DIVOT_ONLY and int(cls) != divot_id:
                            continue
                        preds.append((int(cls), [x, y, w, h], conf))

        matched_gts = set()

        # ── 예측 박스: TP(파랑)/FP(빨강) ──
        for cls, box, conf in preds:
            matched = False
            for gi, (g_cls, g_box) in enumerate(gt_boxes):
                if g_cls != cls or gi in matched_gts:
                    continue
                if iou(box, g_box) > MATCH_IOU:
                    matched = True
                    matched_gts.add(gi)
                    break

            cx, cy, bw, bh = box
            x1, y1, x2, y2 = int((cx-bw/2)*W), int((cy-bh/2)*H), int((cx+bw/2)*W), int((cy+bh/2)*H)
            if matched:
                color, label = (255, 0, 0), f"TP-{CLASS_NAMES[cls]} {conf:.2f}"   # 파랑
            else:
                color, label = (0, 0, 255), f"FP-{CLASS_NAMES[cls]} {conf:.2f}"   # 빨강
            cv2.rectangle(img, (x1, y1), (x2, y2), color, thick)
            cv2.putText(img, label, (x1, max(y1-5, 10)), cv2.FONT_HERSHEY_SIMPLEX,
                        VIZ_FONT_SCALE, color, VIZ_TEXT_THICK, cv2.LINE_AA)

        # ── GT 박스: 항상 표시 (TP/FN 구분 색상 다르게)
        for gi, (cls, box) in enumerate(gt_boxes):
            cx, cy, bw, bh = box
            x1, y1, x2, y2 = int((cx-bw/2)*W), int((cy-bh/2)*H), int((cx+bw/2)*W), int((cy+bh/2)*H)

            if gi in matched_gts:
                # TP에 해당하는 GT → 연한 초록색
                color, label = (0, 200, 0), f"GT(TP)-{CLASS_NAMES[cls]}"
            else:
                # FN (미검출 GT) → 진한 초록색
                color, label = (0, 255, 0), f"GT(FN)-{CLASS_NAMES[cls]}"

            cv2.rectangle(img, (x1, y1), (x2, y2), color, thick)
            cv2.putText(img, label, (x1, max(y1-5, 10)), cv2.FONT_HERSHEY_SIMPLEX,
                        VIZ_FONT_SCALE, color, VIZ_TEXT_THICK, cv2.LINE_AA)

        cv2.imwrite(str(viz_dir / img_path.name), img)


    print(f"✅ VIZ 완료 — 저장 경로: {viz_dir}")


### 3모델 결과 2

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# ============================================================
# YOLOv8 TensorRT — Multi-Seed (No Groups)
# Parallel Predict → Global Ensemble → mAP (val@0.001 clone) & AUROC
# - 실행마다 고유 폴더 생성
# - per-image 로그 비표시
# - FN(미검출)을 AUROC에 점수 0 양성으로 반영
# - mAP clone과 동일하게 IoU 비교는 '>' 사용
# - (옵션) 앙상블 결과를 원본 이미지에 시각화/저장 (전 클래스)
# - (추가) Divot 전용 ROC 표/그래프 저장
# ============================================================

# ── [무음 설정] ultralytics 임포트 전에 ──
import os, logging, math, gc, random, csv
os.environ["ULTRALYTICS_QUIET"] = "1"   # Ultralytics 내부 로그 억제
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True,max_split_size_mb:128")

from concurrent.futures import ThreadPoolExecutor, as_completed
from ultralytics import YOLO
from ultralytics.utils import LOGGER
from ultralytics.utils.metrics import ap_per_class as _ap_per_class
from pathlib import Path
from tqdm import tqdm
import numpy as np
import torch
import yaml
from datetime import datetime
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import cv2  # ← 시각화용

# ── Ultralytics 로거 레벨 하향 ──
LOGGER.setLevel(logging.ERROR)
torch.set_num_threads(1)

# ==============================
# 공통 설정
# ==============================
TEST_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251113_c/images/test")
GT_LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251113_c/labels/test")
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251113_c/data.yaml"

# ---- 실행마다 고유 폴더 생성 ----
BASE_ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/runs/predict_eval/multimodel_ensemble_eval_no_groups")
RUN_NAME = f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
BASE_DIR = BASE_ROOT / RUN_NAME
BASE_DIR.mkdir(parents=True, exist_ok=True)

# 입력 해상도(정규화 복원용) — (주의) 시각화는 실제 이미지 W,H를 사용함
IMG_W, IMG_H = 1920, 1536

# Predict 설정
IMG_SIZE       = 640
DEVICE         = 0
PRED_CONF      = 0.001     # 저장용 conf (mAP clone은 conf 컷 없음)
PRED_IOU_NMS   = 0.5
AUGMENT        = False
AGNOSTIC_NMS   = True
RECT           = False
MAX_WORKERS    = 4

# Ensemble 설정
IOU_MERGE          = 0.65   # IoU ≥ IOU_MERGE → 동일 객체 간주
CONF_THRESH_SINGLE = 0.65  # 단일 매칭 채택 하한
MIN_MATCH          = 2     # 매칭된 시드 수

# Evaluation 설정
IOU_THRS = np.linspace(0.5, 0.95, 10, dtype=np.float32)  # mAP@0.5:0.95
MATCH_IOU = 0.5   # AUROC/TPFPFN 매칭 임계치 (mAP clone과 동일하게 '>')

# ===[시각화 토글/옵션 (앙상블)]====================================
VIZ_SAVE           = False         # ← True면 앙상블 결과를 원본 이미지에 그려 저장
VIZ_USE_PRED_CONF  = False           # ← True면 시각화 임계값을 PRED_CONF와 동일하게 사용
VIZ_SCORE_MIN      = 0.001           # ← VIZ_USE_PRED_CONF=False일 때 쓰는 별도 임계값
VIZ_OUTDIR_NAME    = "viz"          # 출력 폴더명 (all_ensembles/ 아래에 생성)
VIZ_DIVOT_ONLY     = True      # ✅ True면 Divot만 시각화, False면 전 클래스 표시
# 스타일
VIZ_THICK_BASE     = 2
VIZ_FONT_SCALE     = 0.5
VIZ_TEXT_THICK     = 1

# ===[Divot 전용 ROC 테이블/그래프]===============================
DRAW_DIVOT_ROC     = True
TARGET_CLASS_NAME  = "Divot"
# 기존: ROC_THRESH_GRID = np.linspace(0.0, 1.0, 11)
# 수정 ↓↓↓
ROC_THRESH_GRID    = np.concatenate(([0.001], np.linspace(0.0, 1.0, 11)))
ROC_OUTDIR_NAME    = "roc_divot"
# ================================================================

# ==============================
# 시드별 엔진 (그룹 없음)
# ==============================
ENGINES = {
    "best1": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/best_fp16_bs1_640px_static.engine",
    "best2": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/precision_BEST_epoch70_0.95995_fp16_bs1_640px_static.engine",
    "best3": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/best_fp16_bs1_640px_static.engine",
}
# ==============================
# 유틸
# ==============================
def load_yolo_txt(path: Path):
    if (path is None) or (not path.exists()):
        return np.zeros((0, 6), dtype=np.float32)
    rows = []
    with open(path, "r") as f:
        for line in f:
            vals = list(map(float, line.strip().split()))
            if len(vals) == 5:  # GT
                c, x, y, w, h = vals; conf = 1.0
            else:               # Pred
                c, x, y, w, h, conf = vals
            rows.append([c, x, y, w, h, conf])
    return np.asarray(rows, dtype=np.float32)

def xywh2xyxy(boxes, W=IMG_W, H=IMG_H):
    if len(boxes) == 0: return np.zeros((0, 4), dtype=np.float32)
    x, y, w, h = boxes.T
    x1 = (x - w/2) * W; y1 = (y - h/2) * H
    x2 = (x + w/2) * W; y2 = (y + h/2) * H
    return np.stack([x1, y1, x2, y2], axis=1).astype(np.float32)

def xyxy2xywh(boxes, W=IMG_W, H=IMG_H):
    if len(boxes) == 0: return np.zeros((0, 4), dtype=np.float32)
    x1, y1, x2, y2 = boxes.T
    x = (x1 + x2) / 2 / W; y = (y1 + y2) / 2 / H
    w = (x2 - x1) / W;     h = (y2 - y1) / H
    return np.stack([x, y, w, h], axis=1).astype(np.float32)

def box_iou_np(a, b):
    if len(a) == 0 or len(b) == 0:
        return np.zeros((len(a), len(b)), dtype=np.float32)
    area1 = (a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1])
    area2 = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
    ix1 = np.maximum(a[:, None, 0], b[:, 0]); iy1 = np.maximum(a[:, None, 1], b[:, 1])
    ix2 = np.minimum(a[:, None, 2], b[:, 2]); iy2 = np.minimum(a[:, None, 3], b[:, 3])
    iw = np.clip(ix2 - ix1, 0, None); ih = np.clip(iy2 - iy1, 0, None)
    inter = iw * ih
    return inter / (area1[:, None] + area2 - inter + 1e-6)

def stable_conf_sort(conf: np.ndarray):
    idx = np.arange(len(conf))
    return np.lexsort((idx, -conf))

def cat_or_empty(arrs, axis=0, empty_shape=None, dtype=None):
    arrs = [a for a in arrs if a is not None]
    if len(arrs) == 0:
        return np.zeros(empty_shape, dtype=dtype)
    try:
        return np.concatenate(arrs, axis=axis)
    except Exception:
        return np.zeros(empty_shape, dtype=dtype)

def ap_per_class_compat(tp, conf, pred_cls, target_cls):
    out = _ap_per_class(tp, conf, pred_cls, target_cls)
    if len(out) == 5:
        p, r, ap, f1, ap_class = out
    elif len(out) >= 7:
        p, r, ap, f1, ap_class = out[2], out[3], out[5], out[4], out[6]
    else:
        raise RuntimeError("Unexpected ap_per_class return format")
    return p, r, ap, f1, ap_class

def xywhn2xyxy_pixels(xywhn, W, H):
    if len(xywhn) == 0:
        return np.zeros((0, 4), dtype=np.float32)
    x, y, w, h = xywhn.T
    x1 = (x - w/2) * W
    y1 = (y - h/2) * H
    x2 = (x + w/2) * W
    y2 = (y + h/2) * H
    return np.stack([x1, y1, x2, y2], axis=1).astype(np.float32)

def _class_colors(n):
    colors = []
    for i in range(n):
        random.seed(i + 12345)
        colors.append((random.randint(60,255), random.randint(60,255), random.randint(60,255)))
    return colors

# ==============================
# 클래스 로드
# ==============================
with open(DATA_YAML, "r") as f:
    data_cfg = yaml.safe_load(f)
CLASS_NAMES = data_cfg["names"]
NUM_CLASSES = len(CLASS_NAMES)

# ==============================
# 1) Predict (스레드 병렬, 기본=순차)
# ==============================
print(f"🚀 Parallel inference for seeds (no groups) ... → {BASE_DIR}")
predict_jobs = []  # (seed, out_label_dir)

def _predict_one(seed_name, engine_path):
    out_dir = BASE_DIR / "raw_preds" / seed_name
    out_dir.mkdir(parents=True, exist_ok=True)
    model = YOLO(engine_path)
    model.predict(
        source=str(TEST_DIR),
        imgsz=IMG_SIZE,
        conf=PRED_CONF,
        iou=PRED_IOU_NMS,
        device=DEVICE,
        rect=RECT,
        agnostic_nms=AGNOSTIC_NMS,
        augment=AUGMENT,
        deterministic=True,
        seed=1,
        save_txt=True,
        save_conf=True,
        project=out_dir,
        name="pred_labels",
        exist_ok=True,
        verbose=False,
    )
    try:
        model.predictor = None
    except Exception:
        pass
    del model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.synchronize(); torch.cuda.empty_cache()
    return out_dir / "pred_labels" / "labels"

futures = []
with ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(ENGINES))) as ex:
    for sname, eng in ENGINES.items():
        futures.append(ex.submit(_predict_one, sname, eng))
    for fut in tqdm(as_completed(futures), total=len(futures), desc="Predicting"):
        try:
            label_dir = fut.result()
            predict_jobs.append((Path(label_dir).name, label_dir))
            print(f"✅ {label_dir}")
        except Exception as e:
            print("❌ Predict failed:", e)

print(f"🎯 총 {len(predict_jobs)}개 시드 예측 완료")
if not predict_jobs:
    raise SystemExit("No predictions were produced. Check engines/paths.")

# ==============================
# 2) 글로벌 앙상블 (모든 시드 합침)
# ==============================
def ensemble_merge(label_dirs, out_dir: Path,
                   iou_thr=IOU_MERGE, min_match=MIN_MATCH, conf_single=CONF_THRESH_SINGLE):
    out_dir.mkdir(parents=True, exist_ok=True)
    base = label_dirs[0]
    all_files = sorted(os.listdir(base))

    for file in tqdm(all_files, desc=f"Ensembling → {out_dir.name}"):
        preds_all = [load_yolo_txt(Path(d) / file) for d in label_dirs]
        combined = np.concatenate(preds_all, axis=0) if len(preds_all) else np.zeros((0, 6), dtype=np.float32)
        if len(combined) == 0:
            open(out_dir / file, "w").close()
            continue

        boxes_xyxy = xywh2xyxy(combined[:, 1:5])
        cls = combined[:, 0].astype(int)
        conf = combined[:, 5].astype(np.float32)

        used = np.zeros(len(combined), dtype=bool)
        final_preds = []

        for i in range(len(combined)):
            if used[i]:
                continue
            same_cls_idx = np.where(cls == cls[i])[0]
            ious = box_iou_np(boxes_xyxy[i:i+1], boxes_xyxy[same_cls_idx])[0]
            matched = same_cls_idx[ious >= iou_thr]
            used[matched] = True

            k = len(matched)
            if k >= min_match:
                avg_box = boxes_xyxy[matched].mean(axis=0)
                avg_conf = conf[matched].mean()
                final_preds.append([cls[i], *avg_box, avg_conf])
            elif k == 1 and conf[i] >= conf_single:
                final_preds.append([cls[i], *boxes_xyxy[i], conf[i]])

        if len(final_preds):
            final_preds = np.array(final_preds, dtype=np.float32)
            xywh = xyxy2xywh(final_preds[:, 1:5])
            out = np.column_stack([final_preds[:, 0], xywh, final_preds[:, 5]])
            np.savetxt(out_dir / file, out, fmt="%.6f")
        else:
            open(out_dir / file, "w").close()

ALL_LABEL_DIRS = [ld for _, ld in predict_jobs]
ALL_ENSEMBLE_DIR = BASE_DIR / "all_ensembles" / "labels"
ensemble_merge(ALL_LABEL_DIRS, ALL_ENSEMBLE_DIR)
print(f"✅ Global ensemble saved: {ALL_ENSEMBLE_DIR}")

# ==============================
# 3) mAP clone(val@0.001) + AUROC + Divot ROC
# ==============================
def evaluate_map_auroc(PRED_LABEL_DIR: Path, title: str):
    print(f"\n================= ✅ EVALUATION — {title} =================")
    gt_files = sorted(GT_LABEL_DIR.glob("*.txt"))
    pred_map = {p.stem: p for p in PRED_LABEL_DIR.glob("*.txt")}

    # ----- mAP clone -----
    tp_list, conf_list, pred_cls_list, target_cls_list = [], [], [], []
    for gf in tqdm(gt_files, desc="Collecting stats for mAP..."):
        stem = gf.stem
        pf = pred_map.get(stem, None)

        preds = load_yolo_txt(pf)
        gts   = load_yolo_txt(gf)

        gt_boxes   = xywh2xyxy(gts[:, 1:5]) if len(gts) else np.zeros((0, 4), dtype=np.float32)
        gt_cls     = gts[:, 0].astype(int)  if len(gts) else np.zeros((0,), dtype=int)
        pred_boxes = xywh2xyxy(preds[:, 1:5]) if len(preds) else np.zeros((0, 4), dtype=np.float32)
        pred_cls   = preds[:, 0].astype(int) if len(preds) else np.zeros((0,), dtype=int)
        conf       = preds[:, 5].astype(np.float32) if len(preds) else np.zeros((0,), dtype=np.float32)

        correct = np.zeros((len(preds), len(IOU_THRS)), dtype=bool)
        if len(preds) and len(gts):
            ious = box_iou_np(pred_boxes, gt_boxes)
            order = stable_conf_sort(conf)
            pred_boxes, pred_cls, conf = pred_boxes[order], pred_cls[order], conf[order]
            ious = ious[order]
            for j, thr in enumerate(IOU_THRS):
                detected = []
                for k, pc in enumerate(pred_cls):
                    best_iou, best_gt = 0.0, -1
                    for gi, gc in enumerate(gt_cls):
                        if gc != pc or gi in detected:
                            continue
                        iou_val = ious[k, gi]
                        if iou_val > best_iou:
                            best_iou, best_gt = iou_val, gi
                    if best_iou > thr:   # val과 동일: >
                        correct[k, j] = True
                        detected.append(best_gt)

        tp_list.append(correct)
        conf_list.append(conf)
        pred_cls_list.append(pred_cls)
        target_cls_list.append(gt_cls)

    tp             = cat_or_empty(tp_list, axis=0, empty_shape=(0, len(IOU_THRS)), dtype=bool)
    conf_all       = cat_or_empty(conf_list, axis=0, empty_shape=(0,), dtype=np.float32)
    pred_cls_all   = cat_or_empty(pred_cls_list, axis=0, empty_shape=(0,), dtype=int)
    target_cls_all = cat_or_empty(target_cls_list, axis=0, empty_shape=(0,), dtype=int)

    p, r, ap, f1, ap_class = ap_per_class_compat(tp, conf_all, pred_cls_all, target_cls_all)

    # 안전 캐스팅
    p_arr = np.asarray(p, dtype=float).reshape(-1)
    r_arr = np.asarray(r, dtype=float).reshape(-1)
    ap_np = np.asarray(ap, dtype=float)
    if ap_np.ndim == 1:
        n_iou = len(IOU_THRS)
        assert ap_np.size % n_iou == 0, "ap shape unexpected; cannot reshape"
        ap_np = ap_np.reshape(-1, n_iou)

    mAP50   = float(ap_np[:, 0].mean()) if ap_np.size else 0.0
    mAP5095 = float(ap_np.mean()) if ap_np.size else 0.0
    ap_dict = {
        int(c): (float(p_arr[i]) if i < p_arr.size else 0.0,
                 float(r_arr[i]) if i < r_arr.size else 0.0,
                 float(ap_np[i, 0]) if i < ap_np.shape[0] else 0.0,
                 float(ap_np[i].mean()) if i < ap_np.shape[0] else 0.0)
        for i, c in enumerate(ap_class)
    }

    print(f"{'all':<15}  mP={p_arr.mean():.3f}  mR={r_arr.mean():.3f}  mAP50={mAP50:.3f}  mAP50-95={mAP5095:.3f}")
    for i in range(NUM_CLASSES):
        if i in ap_dict:
            pi, ri, ap50_i, ap5095_i = ap_dict[i]
        else:
            pi, ri, ap50_i, ap5095_i = 0.0, 0.0, 0.0, 0.0
        print(f"{CLASS_NAMES[i]:<15}  P={pi:.3f}  R={ri:.3f}  mAP50={ap50_i:.3f}  mAP50-95={ap5095_i:.3f}")

    # ----- AUROC (FN을 점수 0으로 반영) -----
    per_true  = {i: [] for i in range(NUM_CLASSES)}
    per_score = {i: [] for i in range(NUM_CLASSES)}
    tp_cnt = np.zeros(NUM_CLASSES, dtype=int)
    fp_cnt = np.zeros(NUM_CLASSES, dtype=int)
    fn_cnt = np.zeros(NUM_CLASSES, dtype=int)
    gt_cnt = np.zeros(NUM_CLASSES, dtype=int)

    for gf in tqdm(gt_files, desc="Matching for AUROC..."):
        stem = gf.stem
        pf = pred_map.get(stem, None)

        preds = load_yolo_txt(pf)
        gts   = load_yolo_txt(gf)

        if len(gts):
            for c in gts[:, 0].astype(int):
                gt_cnt[c] += 1

        if len(preds) == 0 and len(gts) > 0:
            for c in gts[:, 0].astype(int):
                fn_cnt[c] += 1
                per_true[c].append(1)
                per_score[c].append(0.0)
            continue

        gt_boxes   = xywh2xyxy(gts[:, 1:5]) if len(gts) else np.zeros((0, 4), dtype=np.float32)
        gt_cls     = gts[:, 0].astype(int)  if len(gts) else np.zeros((0,), dtype=int)
        pred_boxes = xywh2xyxy(preds[:, 1:5]) if len(preds) else np.zeros((0, 4), dtype=np.float32)
        pred_cls   = preds[:, 0].astype(int) if len(preds) else np.zeros((0,), dtype=int)
        conf       = preds[:, 5].astype(np.float32) if len(preds) else np.zeros((0,), dtype=np.float32)

        if len(preds) and len(gts):
            ious = box_iou_np(pred_boxes, gt_boxes)
            order = stable_conf_sort(conf)
            pred_boxes, pred_cls, conf = pred_boxes[order], pred_cls[order], conf[order]
            ious = ious[order]

            used = set()
            for k, (pc, cf) in enumerate(zip(pred_cls, conf)):
                best_iou, best_gt = 0.0, -1
                for gi, gc in enumerate(gt_cls):
                    if gc != pc or gi in used:
                        continue
                    iou_val = ious[k, gi]
                    if iou_val > best_iou:
                        best_iou, best_gt = iou_val, gi
                if best_iou > MATCH_IOU and best_gt != -1:
                    tp_cnt[pc] += 1
                    per_true[pc].append(1)
                    per_score[pc].append(cf)
                    used.add(best_gt)
                else:
                    fp_cnt[pc] += 1
                    per_true[pc].append(0)
                    per_score[pc].append(cf)

            for gi, gc in enumerate(gt_cls):
                if gi not in used:
                    fn_cnt[gc] += 1
                    per_true[gc].append(1)
                    per_score[gc].append(0.0)
        elif len(preds) > 0 and len(gts) == 0:
            for c, cf in zip(pred_cls, conf):
                fp_cnt[c] += 1
                per_true[c].append(0)
                per_score[c].append(cf)

    # ── Divot 전용 ROC 저장 ──
    if DRAW_DIVOT_ROC and TARGET_CLASS_NAME in CLASS_NAMES:
        divot_id = CLASS_NAMES.index(TARGET_CLASS_NAME)
        y_true  = np.asarray(per_true[divot_id], dtype=np.float32)
        y_score = np.asarray(per_score[divot_id], dtype=np.float32)

        roc_dir = BASE_DIR / ROC_OUTDIR_NAME
        roc_dir.mkdir(parents=True, exist_ok=True)

        # ROC 곡선
        if len(y_true) >= 2 and len(np.unique(y_true)) >= 2:
            fpr, tpr, _ = roc_curve(y_true, y_score)
            roc_auc = auc(fpr, tpr)
        else:
            fpr, tpr, roc_auc = np.array([0,1]), np.array([0,1]), float("nan")

        plt.figure(figsize=(5,5))
        plt.plot(fpr, tpr, marker='.', linewidth=1, label=f'ROC (AUC={roc_auc:.4f})')
        plt.plot([0,1],[0,1],'--', linewidth=1, label='chance')
        plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title(f'ROC — {TARGET_CLASS_NAME}')
        plt.legend(); plt.grid(True, alpha=0.3)
        fig_path = roc_dir / "divot_roc_curve.png"
        plt.savefig(fig_path, bbox_inches="tight", dpi=150)
        plt.close()
        print(f"[ROC] Divot ROC curve saved: {fig_path}")

        # 임계값 스윕 표
        rows = []
        P = int((y_true == 1).sum())
        N = int((y_true == 0).sum())
        for thr in ROC_THRESH_GRID:
            y_pred = (y_score >= thr).astype(np.int32)
            TP = int(((y_pred == 1) & (y_true == 1)).sum())
            FP = int(((y_pred == 1) & (y_true == 0)).sum())
            TN = int(((y_pred == 0) & (y_true == 0)).sum())
            FN = int(((y_pred == 0) & (y_true == 1)).sum())
            TPR = (TP / P) if P > 0 else 0.0
            FPR = (FP / N) if N > 0 else 0.0
            rows.append((thr, TP, FP, TN, FN, TPR, FPR))
        rows.sort(key=lambda x: x[0])

        csv_path = roc_dir / "divot_roc_table.csv"
        with open(csv_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["threshold", "TP", "FP", "TN", "FN", "TPR", "FPR"])
            writer.writerows(rows)
        print(f"[ROC] Divot ROC table saved: {csv_path}")

        # 콘솔 프리뷰
        print("\n[ROC] Divot — threshold sweep preview")
        print("thr     TP    FP    TN    FN    TPR      FPR")
        for r in rows[:11]:
            print(f"{r[0]:.3f}  {r[1]:5d} {r[2]:5d} {r[3]:5d} {r[4]:5d}  {r[5]:.4f}  {r[6]:.4f}")

    # ── AUROC 표준 출력 ──
    print("\n📈 AUROC Evaluation per class")
    print("─────────────────────────────────────────────────────────────")
    aurocs = []
    for i in range(NUM_CLASSES):
        y_true_i  = np.asarray(per_true[i], dtype=np.float32)
        y_score_i = np.asarray(per_score[i], dtype=np.float32)
        if len(y_true_i) < 2 or len(np.unique(y_true_i)) < 2:
            auroc_i = float("nan")
        else:
            try:
                auroc_i = roc_auc_score(y_true_i, y_score_i)
            except ValueError:
                auroc_i = float("nan")
        aurocs.append(auroc_i)
        auc_str = f"{auroc_i:.4f}" if not math.isnan(auroc_i) else "NaN"
        print(f"{CLASS_NAMES[i]:<18s}: AUROC={auc_str:<8s} | "
              f"TP={tp_cnt[i]:<4d} FP={fp_cnt[i]:<4d} FN={fn_cnt[i]:<4d} GT={gt_cnt[i]:<4d}")
    print("─────────────────────────────────────────────────────────────")
    mean_auroc = np.nanmean(aurocs) if np.any(~np.isnan(aurocs)) else float("nan")
    if not math.isnan(mean_auroc):
        print(f"🔹 Mean AUROC (macro): {mean_auroc:.4f}")
    else:
        print("🔹 Mean AUROC: NaN")
    print("======================================================================")

# 실행 (글로벌만)
evaluate_map_auroc(ALL_ENSEMBLE_DIR, title=f"Global Ensemble — ALL SEEDS ({RUN_NAME})")

# ==============================
# 4) (옵션) 앙상블 시각화/저장 — GT/TP/FP 구분 + Divot 전용 모드
# ==============================
if VIZ_SAVE:
    viz_dir = BASE_DIR / "all_ensembles" / VIZ_OUTDIR_NAME
    viz_dir.mkdir(parents=True, exist_ok=True)
    thr = PRED_CONF if VIZ_USE_PRED_CONF else VIZ_SCORE_MIN

    print(f"\n🖼  Drawing ENSEMBLE predictions (GT=green, TP=blue, FP=red)"
          f"{' [DIVOT ONLY]' if VIZ_DIVOT_ONLY else ''} → {viz_dir} (thr={thr})")

    img_paths = [p for p in sorted(TEST_DIR.glob("*"))
                 if p.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}]

    def iou(box1, box2):
        x1_min, y1_min = box1[0]-box1[2]/2, box1[1]-box1[3]/2
        x1_max, y1_max = box1[0]+box1[2]/2, box1[1]+box1[3]/2
        x2_min, y2_min = box2[0]-box2[2]/2, box2[1]-box2[3]/2
        x2_max, y2_max = box2[0]+box2[2]/2, box2[1]+box2[3]/2
        inter_w = max(0, min(x1_max, x2_max) - max(x1_min, x2_min))
        inter_h = max(0, min(y1_max, y2_max) - max(y1_min, y2_min))
        inter_area = inter_w * inter_h
        union_area = box1[2]*box1[3] + box2[2]*box2[3] - inter_area
        return inter_area / union_area if union_area > 0 else 0.0

    # Divot 클래스 ID 찾기
    divot_id = None
    if "Divot" in CLASS_NAMES:
        divot_id = CLASS_NAMES.index("Divot")

    for img_path in tqdm(img_paths, desc="Drawing (GT/TP/FP)"):
        stem = img_path.stem
        gt_path = GT_LABEL_DIR / f"{stem}.txt"
        pred_path = ALL_ENSEMBLE_DIR / f"{stem}.txt"
        if not gt_path.exists() and not pred_path.exists():
            continue

        img = cv2.imread(str(img_path))
        if img is None:
            continue
        H, W = img.shape[:2]
        thick = max(1, int(VIZ_THICK_BASE * (W + H) / 2000))

        # ── GT 로드 ──
        gt_boxes = []
        if gt_path.exists():
            with open(gt_path, "r") as f:
                for line in f:
                    vals = list(map(float, line.strip().split()))
                    if len(vals) >= 5:
                        cls = int(vals[0])
                        if VIZ_DIVOT_ONLY and cls != divot_id:
                            continue
                        gt_boxes.append((cls, vals[1:5]))

        # ── 예측 로드 ──
        preds = []
        if pred_path.exists():
            with open(pred_path, "r") as f:
                for line in f:
                    vals = list(map(float, line.strip().split()))
                    if len(vals) >= 6:
                        cls, x, y, w, h, conf = vals
                        if conf < thr:
                            continue
                        if VIZ_DIVOT_ONLY and int(cls) != divot_id:
                            continue
                        preds.append((int(cls), [x, y, w, h], conf))

        matched_gts = set()

        # ── 예측 박스: TP(파랑)/FP(빨강) ──
        for cls, box, conf in preds:
            matched = False
            for gi, (g_cls, g_box) in enumerate(gt_boxes):
                if g_cls != cls or gi in matched_gts:
                    continue
                if iou(box, g_box) > MATCH_IOU:
                    matched = True
                    matched_gts.add(gi)
                    break

            cx, cy, bw, bh = box
            x1, y1, x2, y2 = int((cx-bw/2)*W), int((cy-bh/2)*H), int((cx+bw/2)*W), int((cy+bh/2)*H)
            if matched:
                color, label = (255, 0, 0), f"TP-{CLASS_NAMES[cls]} {conf:.2f}"   # 파랑
            else:
                color, label = (0, 0, 255), f"FP-{CLASS_NAMES[cls]} {conf:.2f}"   # 빨강
            cv2.rectangle(img, (x1, y1), (x2, y2), color, thick)
            cv2.putText(img, label, (x1, max(y1-5, 10)), cv2.FONT_HERSHEY_SIMPLEX,
                        VIZ_FONT_SCALE, color, VIZ_TEXT_THICK, cv2.LINE_AA)

        # ── GT 박스: 항상 표시 (TP/FN 구분 색상 다르게)
        for gi, (cls, box) in enumerate(gt_boxes):
            cx, cy, bw, bh = box
            x1, y1, x2, y2 = int((cx-bw/2)*W), int((cy-bh/2)*H), int((cx+bw/2)*W), int((cy+bh/2)*H)

            if gi in matched_gts:
                # TP에 해당하는 GT → 연한 초록색
                color, label = (0, 200, 0), f"GT(TP)-{CLASS_NAMES[cls]}"
            else:
                # FN (미검출 GT) → 진한 초록색
                color, label = (0, 255, 0), f"GT(FN)-{CLASS_NAMES[cls]}"

            cv2.rectangle(img, (x1, y1), (x2, y2), color, thick)
            cv2.putText(img, label, (x1, max(y1-5, 10)), cv2.FONT_HERSHEY_SIMPLEX,
                        VIZ_FONT_SCALE, color, VIZ_TEXT_THICK, cv2.LINE_AA)

        cv2.imwrite(str(viz_dir / img_path.name), img)


    print(f"✅ VIZ 완료 — 저장 경로: {viz_dir}")


### 평균치 + 비중변경

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# ============================================================
# YOLOv8 TensorRT — Multi-Seed (No Groups)
# Parallel Predict → Global Ensemble → mAP (val@0.001 clone) & AUROC
# - 실행마다 고유 폴더 생성
# - per-image 로그 비표시
# - FN(미검출)을 AUROC에 점수 0 양성으로 반영
# - mAP clone과 동일하게 IoU 비교는 '>' 사용
# - (옵션) 앙상블 결과를 원본 이미지에 시각화/저장 (전 클래스)
# - (추가) Divot 전용 ROC 표/그래프 저장
# ============================================================

# ── [무음 설정] ultralytics 임포트 전에 ──
import os, logging, math, gc, random, csv
os.environ["ULTRALYTICS_QUIET"] = "1"   # Ultralytics 내부 로그 억제
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True,max_split_size_mb:128")

from concurrent.futures import ThreadPoolExecutor, as_completed
from ultralytics import YOLO
from ultralytics.utils import LOGGER
from ultralytics.utils.metrics import ap_per_class as _ap_per_class
from pathlib import Path
from tqdm import tqdm
import numpy as np
import torch
import yaml
from datetime import datetime
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import cv2  # ← 시각화용

# ── Ultralytics 로거 레벨 하향 ──
LOGGER.setLevel(logging.ERROR)
torch.set_num_threads(1)

# ==============================
# 공통 설정
# ==============================
TEST_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251112/images/test")
GT_LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251112/labels/test")
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251112/data.yaml"

# ---- 실행마다 고유 폴더 생성 ----
BASE_ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/runs/predict_eval/multimodel_ensemble_eval_no_groups")
RUN_NAME = f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
BASE_DIR = BASE_ROOT / RUN_NAME
BASE_DIR.mkdir(parents=True, exist_ok=True)

# 입력 해상도(정규화 복원용) — (주의) 시각화는 실제 이미지 W,H를 사용함
IMG_W, IMG_H = 1920, 1536

# Predict 설정
IMG_SIZE       = 640
DEVICE         = 0
PRED_CONF      = 0.001     # 저장용 conf (mAP clone은 conf 컷 없음)
PRED_IOU_NMS   = 0.5
AUGMENT        = False
AGNOSTIC_NMS   = True
RECT           = False
MAX_WORKERS    = 4

# Ensemble 설정
IOU_MERGE          = 0.7   # IoU ≥ IOU_MERGE → 동일 객체 간주
CONF_THRESH_SINGLE = 0.5   # 단일 매칭 채택 하한
MIN_MATCH          = 2     # 매칭된 시드 수

# Evaluation 설정
IOU_THRS = np.linspace(0.5, 0.95, 10, dtype=np.float32)  # mAP@0.5:0.95
MATCH_IOU = 0.5   # AUROC/TPFPFN 매칭 임계치 (mAP clone과 동일하게 '>')

# ===[시각화 토글/옵션 (앙상블)]====================================
VIZ_SAVE           = False         # ← True면 앙상블 결과를 원본 이미지에 그려 저장
VIZ_USE_PRED_CONF  = False         # ← True면 시각화 임계값을 PRED_CONF와 동일하게 사용
VIZ_SCORE_MIN      = 0.001         # ← VIZ_USE_PRED_CONF=False일 때 쓰는 별도 임계값
VIZ_OUTDIR_NAME    = "viz"         # 출력 폴더명 (all_ensembles/ 아래에 생성)
VIZ_DIVOT_ONLY     = True          # ✅ True면 Divot만 시각화, False면 전 클래스 표시
# 스타일
VIZ_THICK_BASE     = 2
VIZ_FONT_SCALE     = 0.5
VIZ_TEXT_THICK     = 1

# ===[Divot 전용 ROC 테이블/그래프]===============================
DRAW_DIVOT_ROC     = True
TARGET_CLASS_NAME  = "Divot"
ROC_THRESH_GRID    = np.concatenate(([0.001], np.linspace(0.0, 1.0, 11)))
ROC_OUTDIR_NAME    = "roc_divot"
# ================================================================

# ==============================
# 시드별 엔진 (그룹 없음)
# ==============================
ENGINES = {
    "best1": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/best_fp16_bs1_640px_static.engine",
    "best2": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/precision_BEST_epoch70_0.95995_fp16_bs1_640px_static.engine",
    "best3": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/recall_BEST_epoch35_0.74789_fp16_bs1_640px_static.engine",
}

# ===[Ensemble 가중치]=========================================
WEIGHTS = {
    "best1": 0.8,
    "best2": 0.1,
    "best3": 0.1,
}
# ============================================================

# ==============================
# 유틸
# ==============================
def load_yolo_txt(path: Path):
    if (path is None) or (not path.exists()):
        return np.zeros((0, 6), dtype=np.float32)
    rows = []
    with open(path, "r") as f:
        for line in f:
            vals = list(map(float, line.strip().split()))
            if len(vals) == 5:  # GT
                c, x, y, w, h = vals; conf = 1.0
            else:               # Pred
                c, x, y, w, h, conf = vals
            rows.append([c, x, y, w, h, conf])
    return np.asarray(rows, dtype=np.float32)

def xywh2xyxy(boxes, W=IMG_W, H=IMG_H):
    if len(boxes) == 0: return np.zeros((0, 4), dtype=np.float32)
    x, y, w, h = boxes.T
    x1 = (x - w/2) * W; y1 = (y - h/2) * H
    x2 = (x + w/2) * W; y2 = (y + h/2) * H
    return np.stack([x1, y1, x2, y2], axis=1).astype(np.float32)

def xyxy2xywh(boxes, W=IMG_W, H=IMG_H):
    if len(boxes) == 0: return np.zeros((0, 4), dtype=np.float32)
    x1, y1, x2, y2 = boxes.T
    x = (x1 + x2) / 2 / W; y = (y1 + y2) / 2 / H
    w = (x2 - x1) / W;     h = (y2 - y1) / H
    return np.stack([x, y, w, h], axis=1).astype(np.float32)

def box_iou_np(a, b):
    if len(a) == 0 or len(b) == 0:
        return np.zeros((len(a), len(b)), dtype=np.float32)
    area1 = (a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1])
    area2 = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
    ix1 = np.maximum(a[:, None, 0], b[:, 0]); iy1 = np.maximum(a[:, None, 1], b[:, 1])
    ix2 = np.minimum(a[:, None, 2], b[:, 2]); iy2 = np.minimum(a[:, None, 3], b[:, 3])
    iw = np.clip(ix2 - ix1, 0, None); ih = np.clip(iy2 - iy1, 0, None)
    inter = iw * ih
    return inter / (area1[:, None] + area2 - inter + 1e-6)

def stable_conf_sort(conf: np.ndarray):
    idx = np.arange(len(conf))
    return np.lexsort((idx, -conf))

def cat_or_empty(arrs, axis=0, empty_shape=None, dtype=None):
    arrs = [a for a in arrs if a is not None]
    if len(arrs) == 0:
        return np.zeros(empty_shape, dtype=dtype)
    try:
        return np.concatenate(arrs, axis=axis)
    except Exception:
        return np.zeros(empty_shape, dtype=dtype)

def ap_per_class_compat(tp, conf, pred_cls, target_cls):
    out = _ap_per_class(tp, conf, pred_cls, target_cls)
    if len(out) == 5:
        p, r, ap, f1, ap_class = out
    elif len(out) >= 7:
        p, r, ap, f1, ap_class = out[2], out[3], out[5], out[4], out[6]
    else:
        raise RuntimeError("Unexpected ap_per_class return format")
    return p, r, ap, f1, ap_class

def xywhn2xyxy_pixels(xywhn, W, H):
    if len(xywhn) == 0:
        return np.zeros((0, 4), dtype=np.float32)
    x, y, w, h = xywhn.T
    x1 = (x - w/2) * W
    y1 = (y - h/2) * H
    x2 = (x + w/2) * W
    y2 = (y + h/2) * H
    return np.stack([x1, y1, x2, y2], axis=1).astype(np.float32)

def _class_colors(n):
    colors = []
    for i in range(n):
        random.seed(i + 12345)
        colors.append((random.randint(60,255), random.randint(60,255), random.randint(60,255)))
    return colors

# ==============================
# 클래스 로드
# ==============================
with open(DATA_YAML, "r") as f:
    data_cfg = yaml.safe_load(f)
CLASS_NAMES = data_cfg["names"]
NUM_CLASSES = len(CLASS_NAMES)

# ==============================
# 1) Predict (스레드 병렬, 기본=순차)
# ==============================
print(f"🚀 Parallel inference for seeds (no groups) ... → {BASE_DIR}")
predict_jobs = []  # (seed_name, out_label_dir)

def _predict_one(seed_name, engine_path):
    out_dir = BASE_DIR / "raw_preds" / seed_name
    out_dir.mkdir(parents=True, exist_ok=True)
    model = YOLO(engine_path)
    model.predict(
        source=str(TEST_DIR),
        imgsz=IMG_SIZE,
        conf=PRED_CONF,
        iou=PRED_IOU_NMS,
        device=DEVICE,
        rect=RECT,
        agnostic_nms=AGNOSTIC_NMS,
        augment=AUGMENT,
        deterministic=True,
        seed=1,
        save_txt=True,
        save_conf=True,
        project=out_dir,
        name="pred_labels",
        exist_ok=True,
        verbose=False,
    )
    try:
        model.predictor = None
    except Exception:
        pass
    del model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.synchronize(); torch.cuda.empty_cache()
    # 시드명과 라벨 디렉토리 함께 반환
    return seed_name, out_dir / "pred_labels" / "labels"

futures = []
with ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(ENGINES))) as ex:
    for sname, eng in ENGINES.items():
        futures.append(ex.submit(_predict_one, sname, eng))
    for fut in tqdm(as_completed(futures), total=len(futures), desc="Predicting"):
        try:
            seed_name, label_dir = fut.result()
            predict_jobs.append((seed_name, label_dir))
            print(f"✅ [{seed_name}] {label_dir}")
        except Exception as e:
            print("❌ Predict failed:", e)

print(f"🎯 총 {len(predict_jobs)}개 시드 예측 완료")
if not predict_jobs:
    raise SystemExit("No predictions were produced. Check engines/paths.")

# ==============================
# 2) 글로벌 앙상블 (모든 시드 합침 — 가중 평균)
# ==============================
def ensemble_merge(label_dirs, seed_names, out_dir: Path,
                   iou_thr=IOU_MERGE, min_match=MIN_MATCH, conf_single=CONF_THRESH_SINGLE):
    """
    label_dirs: [str|Path, ...]
    seed_names: [str, ...]  ← WEIGHTS 매핑용 (label_dirs와 같은 순서)
    """
    out_dir.mkdir(parents=True, exist_ok=True)
    base = label_dirs[0]
    all_files = sorted(os.listdir(base))

    # 시드 순서별 weight 벡터 (없으면 균등)
    default_w = 1.0 / max(1, len(seed_names))
    seed_w = np.array([WEIGHTS.get(s, default_w) for s in seed_names], dtype=np.float32)

    for file in tqdm(all_files, desc=f"Ensembling → {out_dir.name}"):
        # 각 시드의 preds와 해당 시드 인덱스 수집
        preds_all = []
        seed_idx_all = []
        for si, (sname, d) in enumerate(zip(seed_names, label_dirs)):
            arr = load_yolo_txt(Path(d) / file)
            if arr is None or len(arr) == 0:
                continue
            preds_all.append(arr)
            seed_idx_all.append(np.full(len(arr), si, dtype=np.int32))

        if len(preds_all) == 0:
            open(out_dir / file, "w").close()
            continue

        combined = np.concatenate(preds_all, axis=0)
        seed_idx = np.concatenate(seed_idx_all, axis=0)  # 각 행이 어느 시드에서 왔는지

        boxes_xyxy = xywh2xyxy(combined[:, 1:5])
        cls = combined[:, 0].astype(int)
        conf = combined[:, 5].astype(np.float32)

        used = np.zeros(len(combined), dtype=bool)
        final_preds = []

        for i in range(len(combined)):
            if used[i]:
                continue
            # 같은 클래스끼리만 매칭
            same_cls_idx = np.where(cls == cls[i])[0]
            ious = box_iou_np(boxes_xyxy[i:i+1], boxes_xyxy[same_cls_idx])[0]
            matched_local = np.where(ious >= iou_thr)[0]
            matched = same_cls_idx[matched_local]
            used[matched] = True

            k = len(matched)
            if k >= min_match:
                # ✅ 가중 평균(박스/컨피던스)
                w = seed_w[seed_idx[matched]]            # 시드별 가중치
                w = w / (w.sum() + 1e-12)                # 정규화
                avg_box = (boxes_xyxy[matched] * w[:, None]).sum(axis=0)
                avg_conf = float((conf[matched] * w).sum())
                final_preds.append([cls[i], *avg_box, avg_conf])
            elif k == 1 and conf[i] >= conf_single:
                # 단일 채택은 그대로
                final_preds.append([cls[i], *boxes_xyxy[i], conf[i]])

        if len(final_preds):
            final_preds = np.array(final_preds, dtype=np.float32)
            xywh = xyxy2xywh(final_preds[:, 1:5])
            out = np.column_stack([final_preds[:, 0], xywh, final_preds[:, 5]])
            np.savetxt(out_dir / file, out, fmt="%.6f")
        else:
            open(out_dir / file, "w").close()

ALL_LABEL_DIRS = [ld for (s, ld) in predict_jobs]
SEED_NAMES     = [s  for (s, ld) in predict_jobs]
ALL_ENSEMBLE_DIR = BASE_DIR / "all_ensembles" / "labels"
ensemble_merge(ALL_LABEL_DIRS, SEED_NAMES, ALL_ENSEMBLE_DIR)
print(f"✅ Global ensemble saved: {ALL_ENSEMBLE_DIR}")

# ==============================
# 3) mAP clone(val@0.001) + AUROC + Divot ROC
# ==============================
def evaluate_map_auroc(PRED_LABEL_DIR: Path, title: str):
    print(f"\n================= ✅ EVALUATION — {title} =================")
    gt_files = sorted(GT_LABEL_DIR.glob("*.txt"))
    pred_map = {p.stem: p for p in PRED_LABEL_DIR.glob("*.txt")}

    # ----- mAP clone -----
    tp_list, conf_list, pred_cls_list, target_cls_list = [], [], [], []
    for gf in tqdm(gt_files, desc="Collecting stats for mAP..."):
        stem = gf.stem
        pf = pred_map.get(stem, None)

        preds = load_yolo_txt(pf)
        gts   = load_yolo_txt(gf)

        gt_boxes   = xywh2xyxy(gts[:, 1:5]) if len(gts) else np.zeros((0, 4), dtype=np.float32)
        gt_cls     = gts[:, 0].astype(int)  if len(gts) else np.zeros((0,), dtype=int)
        pred_boxes = xywh2xyxy(preds[:, 1:5]) if len(preds) else np.zeros((0, 4), dtype=np.float32)
        pred_cls   = preds[:, 0].astype(int) if len(preds) else np.zeros((0,), dtype=int)
        conf       = preds[:, 5].astype(np.float32) if len(preds) else np.zeros((0,), dtype=np.float32)

        correct = np.zeros((len(preds), len(IOU_THRS)), dtype=bool)
        if len(preds) and len(gts):
            ious = box_iou_np(pred_boxes, gt_boxes)
            order = stable_conf_sort(conf)
            pred_boxes, pred_cls, conf = pred_boxes[order], pred_cls[order], conf[order]
            ious = ious[order]
            for j, thr in enumerate(IOU_THRS):
                detected = []
                for k, pc in enumerate(pred_cls):
                    best_iou, best_gt = 0.0, -1
                    for gi, gc in enumerate(gt_cls):
                        if gc != pc or gi in detected:
                            continue
                        iou_val = ious[k, gi]
                        if iou_val > best_iou:
                            best_iou, best_gt = iou_val, gi
                    if best_iou > thr:   # val과 동일: >
                        correct[k, j] = True
                        detected.append(best_gt)

        tp_list.append(correct)
        conf_list.append(conf)
        pred_cls_list.append(pred_cls)
        target_cls_list.append(gt_cls)

    tp             = cat_or_empty(tp_list, axis=0, empty_shape=(0, len(IOU_THRS)), dtype=bool)
    conf_all       = cat_or_empty(conf_list, axis=0, empty_shape=(0,), dtype=np.float32)
    pred_cls_all   = cat_or_empty(pred_cls_list, axis=0, empty_shape=(0,), dtype=int)
    target_cls_all = cat_or_empty(target_cls_list, axis=0, empty_shape=(0,), dtype=int)

    p, r, ap, f1, ap_class = ap_per_class_compat(tp, conf_all, pred_cls_all, target_cls_all)

    # 안전 캐스팅
    p_arr = np.asarray(p, dtype=float).reshape(-1)
    r_arr = np.asarray(r, dtype=float).reshape(-1)
    ap_np = np.asarray(ap, dtype=float)
    if ap_np.ndim == 1:
        n_iou = len(IOU_THRS)
        assert ap_np.size % n_iou == 0, "ap shape unexpected; cannot reshape"
        ap_np = ap_np.reshape(-1, n_iou)

    mAP50   = float(ap_np[:, 0].mean()) if ap_np.size else 0.0
    mAP5095 = float(ap_np.mean()) if ap_np.size else 0.0
    ap_dict = {
        int(c): (float(p_arr[i]) if i < p_arr.size else 0.0,
                 float(r_arr[i]) if i < r_arr.size else 0.0,
                 float(ap_np[i, 0]) if i < ap_np.shape[0] else 0.0,
                 float(ap_np[i].mean()) if i < ap_np.shape[0] else 0.0)
        for i, c in enumerate(ap_class)
    }

    print(f"{'all':<15}  mP={p_arr.mean():.3f}  mR={r_arr.mean():.3f}  mAP50={mAP50:.3f}  mAP50-95={mAP5095:.3f}")
    for i in range(NUM_CLASSES):
        if i in ap_dict:
            pi, ri, ap50_i, ap5095_i = ap_dict[i]
        else:
            pi, ri, ap50_i, ap5095_i = 0.0, 0.0, 0.0, 0.0
        print(f"{CLASS_NAMES[i]:<15}  P={pi:.3f}  R={ri:.3f}  mAP50={ap50_i:.3f}  mAP50-95={ap5095_i:.3f}")

    # ----- AUROC (FN을 점수 0으로 반영) -----
    per_true  = {i: [] for i in range(NUM_CLASSES)}
    per_score = {i: [] for i in range(NUM_CLASSES)}
    tp_cnt = np.zeros(NUM_CLASSES, dtype=int)
    fp_cnt = np.zeros(NUM_CLASSES, dtype=int)
    fn_cnt = np.zeros(NUM_CLASSES, dtype=int)
    gt_cnt = np.zeros(NUM_CLASSES, dtype=int)

    for gf in tqdm(gt_files, desc="Matching for AUROC..."):
        stem = gf.stem
        pf = pred_map.get(stem, None)

        preds = load_yolo_txt(pf)
        gts   = load_yolo_txt(gf)

        if len(gts):
            for c in gts[:, 0].astype(int):
                gt_cnt[c] += 1

        if len(preds) == 0 and len(gts) > 0:
            for c in gts[:, 0].astype(int):
                fn_cnt[c] += 1
                per_true[c].append(1)
                per_score[c].append(0.0)
            continue

        gt_boxes   = xywh2xyxy(gts[:, 1:5]) if len(gts) else np.zeros((0, 4), dtype=np.float32)
        gt_cls     = gts[:, 0].astype(int)  if len(gts) else np.zeros((0,), dtype=int)
        pred_boxes = xywh2xyxy(preds[:, 1:5]) if len(preds) else np.zeros((0, 4), dtype=np.float32)
        pred_cls   = preds[:, 0].astype(int) if len(preds) else np.zeros((0,), dtype=int)
        conf       = preds[:, 5].astype(np.float32) if len(preds) else np.zeros((0,), dtype=np.float32)

        if len(preds) and len(gts):
            ious = box_iou_np(pred_boxes, gt_boxes)
            order = stable_conf_sort(conf)
            pred_boxes, pred_cls, conf = pred_boxes[order], pred_cls[order], conf[order]
            ious = ious[order]

            used = set()
            for k, (pc, cf) in enumerate(zip(pred_cls, conf)):
                best_iou, best_gt = 0.0, -1
                for gi, gc in enumerate(gt_cls):
                    if gc != pc or gi in used:
                        continue
                    iou_val = ious[k, gi]
                    if iou_val > best_iou:
                        best_iou, best_gt = iou_val, gi
                if best_iou > MATCH_IOU and best_gt != -1:
                    tp_cnt[pc] += 1
                    per_true[pc].append(1)
                    per_score[pc].append(cf)
                    used.add(best_gt)
                else:
                    fp_cnt[pc] += 1
                    per_true[pc].append(0)
                    per_score[pc].append(cf)

            for gi, gc in enumerate(gt_cls):
                if gi not in used:
                    fn_cnt[gc] += 1
                    per_true[gc].append(1)
                    per_score[gc].append(0.0)
        elif len(preds) > 0 and len(gts) == 0:
            for c, cf in zip(pred_cls, conf):
                fp_cnt[c] += 1
                per_true[c].append(0)
                per_score[c].append(cf)

    # ── Divot 전용 ROC 저장 ──
    if DRAW_DIVOT_ROC and TARGET_CLASS_NAME in CLASS_NAMES:
        divot_id = CLASS_NAMES.index(TARGET_CLASS_NAME)
        y_true  = np.asarray(per_true[divot_id], dtype=np.float32)
        y_score = np.asarray(per_score[divot_id], dtype=np.float32)

        roc_dir = BASE_DIR / ROC_OUTDIR_NAME
        roc_dir.mkdir(parents=True, exist_ok=True)

        # ROC 곡선
        if len(y_true) >= 2 and len(np.unique(y_true)) >= 2:
            fpr, tpr, _ = roc_curve(y_true, y_score)
            roc_auc = auc(fpr, tpr)
        else:
            fpr, tpr, roc_auc = np.array([0,1]), np.array([0,1]), float("nan")

        plt.figure(figsize=(5,5))
        plt.plot(fpr, tpr, marker='.', linewidth=1, label=f'ROC (AUC={roc_auc:.4f})')
        plt.plot([0,1],[0,1],'--', linewidth=1, label='chance')
        plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title(f'ROC — {TARGET_CLASS_NAME}')
        plt.legend(); plt.grid(True, alpha=0.3)
        fig_path = roc_dir / "divot_roc_curve.png"
        plt.savefig(fig_path, bbox_inches="tight", dpi=150)
        plt.close()
        print(f"[ROC] Divot ROC curve saved: {fig_path}")

        # 임계값 스윕 표
        rows = []
        P = int((y_true == 1).sum())
        N = int((y_true == 0).sum())
        for thr in ROC_THRESH_GRID:
            y_pred = (y_score >= thr).astype(np.int32)
            TP = int(((y_pred == 1) & (y_true == 1)).sum())
            FP = int(((y_pred == 1) & (y_true == 0)).sum())
            TN = int(((y_pred == 0) & (y_true == 0)).sum())
            FN = int(((y_pred == 0) & (y_true == 1)).sum())
            TPR = (TP / P) if P > 0 else 0.0
            FPR = (FP / N) if N > 0 else 0.0
            rows.append((thr, TP, FP, TN, FN, TPR, FPR))
        rows.sort(key=lambda x: x[0])

        csv_path = roc_dir / "divot_roc_table.csv"
        with open(csv_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["threshold", "TP", "FP", "TN", "FN", "TPR", "FPR"])
            writer.writerows(rows)
        print(f"[ROC] Divot ROC table saved: {csv_path}")

        # 콘솔 프리뷰
        print("\n[ROC] Divot — threshold sweep preview")
        print("thr     TP    FP    TN    FN    TPR      FPR")
        for r in rows[:11]:
            print(f"{r[0]:.3f}  {r[1]:5d} {r[2]:5d} {r[3]:5d} {r[4]:5d}  {r[5]:.4f}  {r[6]:.4f}")

    # ── AUROC 표준 출력 ──
    print("\n📈 AUROC Evaluation per class")
    print("─────────────────────────────────────────────────────────────")
    aurocs = []
    for i in range(NUM_CLASSES):
        y_true_i  = np.asarray(per_true[i], dtype=np.float32)
        y_score_i = np.asarray(per_score[i], dtype=np.float32)
        if len(y_true_i) < 2 or len(np.unique(y_true_i)) < 2:
            auroc_i = float("nan")
        else:
            try:
                auroc_i = roc_auc_score(y_true_i, y_score_i)
            except ValueError:
                auroc_i = float("nan")
        aurocs.append(auroc_i)
        auc_str = f"{auroc_i:.4f}" if not math.isnan(auroc_i) else "NaN"
        print(f"{CLASS_NAMES[i]:<18s}: AUROC={auc_str:<8s} | "
              f"TP={tp_cnt[i]:<4d} FP={fp_cnt[i]:<4d} FN={fn_cnt[i]:<4d} GT={gt_cnt[i]:<4d}")
    print("─────────────────────────────────────────────────────────────")
    mean_auroc = np.nanmean(aurocs) if np.any(~np.isnan(aurocs)) else float("nan")
    if not math.isnan(mean_auroc):
        print(f"🔹 Mean AUROC (macro): {mean_auroc:.4f}")
    else:
        print("🔹 Mean AUROC: NaN")
    print("======================================================================")

# 실행 (글로벌만)
evaluate_map_auroc(ALL_ENSEMBLE_DIR, title=f"Global Ensemble — ALL SEEDS ({RUN_NAME})")

# ==============================
# 4) (옵션) 앙상블 시각화/저장 — GT/TP/FP 구분 + Divot 전용 모드
# ==============================
if VIZ_SAVE:
    viz_dir = BASE_DIR / "all_ensembles" / VIZ_OUTDIR_NAME
    viz_dir.mkdir(parents=True, exist_ok=True)
    thr = PRED_CONF if VIZ_USE_PRED_CONF else VIZ_SCORE_MIN

    print(f"\n🖼  Drawing ENSEMBLE predictions (GT=green, TP=blue, FP=red)"
          f"{' [DIVOT ONLY]' if VIZ_DIVOT_ONLY else ''} → {viz_dir} (thr={thr})")

    img_paths = [p for p in sorted(TEST_DIR.glob("*"))
                 if p.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}]

    def iou(box1, box2):
        x1_min, y1_min = box1[0]-box1[2]/2, box1[1]-box1[3]/2
        x1_max, y1_max = box1[0]+box1[2]/2, box1[1]+box1[3]/2
        x2_min, y2_min = box2[0]-box2[2]/2, box2[1]-box2[3]/2
        x2_max, y2_max = box2[0]+box2[2]/2, box2[1]+box2[3]/2
        inter_w = max(0, min(x1_max, x2_max) - max(x1_min, x2_min))
        inter_h = max(0, min(y1_max, y2_max) - max(y1_min, y2_min))
        inter_area = inter_w * inter_h
        union_area = box1[2]*box1[3] + box2[2]*box2[3] - inter_area
        return inter_area / union_area if union_area > 0 else 0.0

    # Divot 클래스 ID 찾기
    divot_id = None
    if "Divot" in CLASS_NAMES:
        divot_id = CLASS_NAMES.index("Divot")

    for img_path in tqdm(img_paths, desc="Drawing (GT/TP/FP)"):
        stem = img_path.stem
        gt_path = GT_LABEL_DIR / f"{stem}.txt"
        pred_path = ALL_ENSEMBLE_DIR / f"{stem}.txt"
        if not gt_path.exists() and not pred_path.exists():
            continue

        img = cv2.imread(str(img_path))
        if img is None:
            continue
        H, W = img.shape[:2]
        thick = max(1, int(VIZ_THICK_BASE * (W + H) / 2000))

        # ── GT 로드 ──
        gt_boxes = []
        if gt_path.exists():
            with open(gt_path, "r") as f:
                for line in f:
                    vals = list(map(float, line.strip().split()))
                    if len(vals) >= 5:
                        cls = int(vals[0])
                        if VIZ_DIVOT_ONLY and (divot_id is not None) and cls != divot_id:
                            continue
                        gt_boxes.append((cls, vals[1:5]))

        # ── 예측 로드 ──
        preds = []
        if pred_path.exists():
            with open(pred_path, "r") as f:
                for line in f:
                    vals = list(map(float, line.strip().split()))
                    if len(vals) >= 6:
                        cls, x, y, w, h, conf = vals
                        if conf < thr:
                            continue
                        if VIZ_DIVOT_ONLY and (divot_id is not None) and int(cls) != divot_id:
                            continue
                        preds.append((int(cls), [x, y, w, h], conf))

        matched_gts = set()

        # ── 예측 박스: TP(파랑)/FP(빨강) ──
        for cls, box, conf in preds:
            matched = False
            for gi, (g_cls, g_box) in enumerate(gt_boxes):
                if g_cls != cls or gi in matched_gts:
                    continue
                if iou(box, g_box) > MATCH_IOU:
                    matched = True
                    matched_gts.add(gi)
                    break

            cx, cy, bw, bh = box
            x1, y1, x2, y2 = int((cx-bw/2)*W), int((cy-bh/2)*H), int((cx+bw/2)*W), int((cy+bh/2)*H)
            if matched:
                color, label = (255, 0, 0), f"TP-{CLASS_NAMES[cls]} {conf:.2f}"   # 파랑
            else:
                color, label = (0, 0, 255), f"FP-{CLASS_NAMES[cls]} {conf:.2f}"   # 빨강
            cv2.rectangle(img, (x1, y1), (x2, y2), color, thick)
            cv2.putText(img, label, (x1, max(y1-5, 10)), cv2.FONT_HERSHEY_SIMPLEX,
                        VIZ_FONT_SCALE, color, VIZ_TEXT_THICK, cv2.LINE_AA)

        # ── GT 박스: 항상 표시 (TP/FN 구분 색상 다르게)
        for gi, (cls, box) in enumerate(gt_boxes):
            cx, cy, bw, bh = box
            x1, y1, x2, y2 = int((cx-bw/2)*W), int((cy-bh/2)*H), int((cx+bw/2)*W), int((cy+bh/2)*H)

            if gi in matched_gts:
                # TP에 해당하는 GT → 연한 초록색
                color, label = (0, 200, 0), f"GT(TP)-{CLASS_NAMES[cls]}"
            else:
                # FN (미검출 GT) → 진한 초록색
                color, label = (0, 255, 0), f"GT(FN)-{CLASS_NAMES[cls]}"

            cv2.rectangle(img, (x1, y1), (x2, y2), color, thick)
            cv2.putText(img, label, (x1, max(y1-5, 10)), cv2.FONT_HERSHEY_SIMPLEX,
                        VIZ_FONT_SCALE, color, VIZ_TEXT_THICK, cv2.LINE_AA)

        cv2.imwrite(str(viz_dir / img_path.name), img)

    print(f"✅ VIZ 완료 — 저장 경로: {viz_dir}")


### 강신뢰기능 추가

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# ============================================================
# YOLOv8 TensorRT — Multi-Seed (No Groups)
# Parallel Predict → Global Ensemble → mAP (val@0.001 clone) & AUROC
# - 실행마다 고유 폴더 생성
# - per-image 로그 비표시
# - FN(미검출)을 AUROC에 점수 0 양성으로 반영
# - mAP clone과 동일하게 IoU 비교는 '>' 사용
# - (옵션) 앙상블 결과를 원본 이미지에 시각화/저장 (전 클래스)
# - (추가) Divot 전용 ROC 표/그래프 저장
# ============================================================

# ── [무음 설정] ultralytics 임포트 전에 ──
import os, logging, math, gc, random, csv
os.environ["ULTRALYTICS_QUIET"] = "1"   # Ultralytics 내부 로그 억제
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True,max_split_size_mb:128")

from concurrent.futures import ThreadPoolExecutor, as_completed
from ultralytics import YOLO
from ultralytics.utils import LOGGER
from ultralytics.utils.metrics import ap_per_class as _ap_per_class
from pathlib import Path
from tqdm import tqdm
import numpy as np
import torch
import yaml
from datetime import datetime
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import cv2  # ← 시각화용

# ── Ultralytics 로거 레벨 하향 ──
LOGGER.setLevel(logging.ERROR)
torch.set_num_threads(1)

# ==============================
# 공통 설정
# ==============================
TEST_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251110/images/test")
GT_LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251110/labels/test")
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251110/data.yaml"

# ---- 실행마다 고유 폴더 생성 ----
BASE_ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/runs/predict_eval/multimodel_ensemble_eval_no_groups")
RUN_NAME = f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
BASE_DIR = BASE_ROOT / RUN_NAME
BASE_DIR.mkdir(parents=True, exist_ok=True)

# 입력 해상도(정규화 복원용) — (주의) 시각화는 실제 이미지 W,H를 사용함
IMG_W, IMG_H = 1920, 1536

# Predict 설정
IMG_SIZE       = 640
DEVICE         = 0
PRED_CONF      = 0.001     # 저장용 conf (mAP clone은 conf 컷 없음)
PRED_IOU_NMS   = 0.5
AUGMENT        = False
AGNOSTIC_NMS   = True
RECT           = False
MAX_WORKERS    = 4

# Ensemble 설정
IOU_MERGE          = 0.7   # IoU ≥ IOU_MERGE → 동일 객체 간주
CONF_THRESH_SINGLE = 0.5  # 단일 매칭 채택 하한
MIN_MATCH          = 2     # 매칭된 시드 수
CONF_HIGH_STRONG   = 0.5   # ✅ 강신뢰 규칙 임계값 추가

# Evaluation 설정
IOU_THRS = np.linspace(0.5, 0.95, 10, dtype=np.float32)  # mAP@0.5:0.95
MATCH_IOU = 0.5   # AUROC/TPFPFN 매칭 임계치 (mAP clone과 동일하게 '>')

# ===[시각화 토글/옵션 (앙상블)]====================================
VIZ_SAVE           = True         # ← True면 앙상블 결과를 원본 이미지에 그려 저장
VIZ_USE_PRED_CONF  = False           # ← True면 시각화 임계값을 PRED_CONF와 동일하게 사용
VIZ_SCORE_MIN      = 0.001           # ← VIZ_USE_PRED_CONF=False일 때 쓰는 별도 임계값
VIZ_OUTDIR_NAME    = "viz"          # 출력 폴더명 (all_ensembles/ 아래에 생성)
VIZ_DIVOT_ONLY     = True      # ✅ True면 Divot만 시각화, False면 전 클래스 표시
# 스타일
VIZ_THICK_BASE     = 2
VIZ_FONT_SCALE     = 0.5
VIZ_TEXT_THICK     = 1

# ===[Divot 전용 ROC 테이블/그래프]===============================
DRAW_DIVOT_ROC     = True
TARGET_CLASS_NAME  = "Divot"
ROC_THRESH_GRID    = np.concatenate(([0.001], np.linspace(0.0, 1.0, 11)))
ROC_OUTDIR_NAME    = "roc_divot"
# ================================================================

# ==============================
# 시드별 엔진 (그룹 없음)
# ==============================
ENGINES = {
    "best1": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/best_fp16_bs1_640px_static.engine",
    "best2": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/precision_BEST_epoch70_0.95995_fp16_bs1_640px_static.engine",
    "best3": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/recall_BEST_epoch35_0.74789_fp16_bs1_640px_static.engine",
}

# ==============================
# 유틸
# ==============================
def load_yolo_txt(path: Path):
    if (path is None) or (not path.exists()):
        return np.zeros((0, 6), dtype=np.float32)
    rows = []
    with open(path, "r") as f:
        for line in f:
            vals = list(map(float, line.strip().split()))
            if len(vals) == 5:  # GT
                c, x, y, w, h = vals; conf = 1.0
            else:               # Pred
                c, x, y, w, h, conf = vals
            rows.append([c, x, y, w, h, conf])
    return np.asarray(rows, dtype=np.float32)

def xywh2xyxy(boxes, W=IMG_W, H=IMG_H):
    if len(boxes) == 0: return np.zeros((0, 4), dtype=np.float32)
    x, y, w, h = boxes.T
    x1 = (x - w/2) * W; y1 = (y - h/2) * H
    x2 = (x + w/2) * W; y2 = (y + h/2) * H
    return np.stack([x1, y1, x2, y2], axis=1).astype(np.float32)

def xyxy2xywh(boxes, W=IMG_W, H=IMG_H):
    if len(boxes) == 0: return np.zeros((0, 4), dtype=np.float32)
    x1, y1, x2, y2 = boxes.T
    x = (x1 + x2) / 2 / W; y = (y1 + y2) / 2 / H
    w = (x2 - x1) / W;     h = (y2 - y1) / H
    return np.stack([x, y, w, h], axis=1).astype(np.float32)

def box_iou_np(a, b):
    if len(a) == 0 or len(b) == 0:
        return np.zeros((len(a), len(b)), dtype=np.float32)
    area1 = (a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1])
    area2 = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
    ix1 = np.maximum(a[:, None, 0], b[:, 0]); iy1 = np.maximum(a[:, None, 1], b[:, 1])
    ix2 = np.minimum(a[:, None, 2], b[:, 2]); iy2 = np.minimum(a[:, None, 3], b[:, 3])
    iw = np.clip(ix2 - ix1, 0, None); ih = np.clip(iy2 - iy1, 0, None)
    inter = iw * ih
    return inter / (area1[:, None] + area2 - inter + 1e-6)

def stable_conf_sort(conf: np.ndarray):
    idx = np.arange(len(conf))
    return np.lexsort((idx, -conf))

def cat_or_empty(arrs, axis=0, empty_shape=None, dtype=None):
    arrs = [a for a in arrs if a is not None]
    if len(arrs) == 0:
        return np.zeros(empty_shape, dtype=dtype)
    try:
        return np.concatenate(arrs, axis=axis)
    except Exception:
        return np.zeros(empty_shape, dtype=dtype)

# ==============================
# 1) Predict (스레드 병렬)
# ==============================
print(f"🚀 Parallel inference for seeds (no groups) ... → {BASE_DIR}")
predict_jobs = []

def _predict_one(seed_name, engine_path):
    out_dir = BASE_DIR / "raw_preds" / seed_name
    out_dir.mkdir(parents=True, exist_ok=True)
    model = YOLO(engine_path)
    model.predict(
        source=str(TEST_DIR),
        imgsz=IMG_SIZE,
        conf=PRED_CONF,
        iou=PRED_IOU_NMS,
        device=DEVICE,
        rect=RECT,
        agnostic_nms=AGNOSTIC_NMS,
        augment=AUGMENT,
        deterministic=True,
        seed=1,
        save_txt=True,
        save_conf=True,
        project=out_dir,
        name="pred_labels",
        exist_ok=True,
        verbose=False,
    )
    try:
        model.predictor = None
    except Exception:
        pass
    del model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.synchronize(); torch.cuda.empty_cache()
    return out_dir / "pred_labels" / "labels"

futures = []
with ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(ENGINES))) as ex:
    for sname, eng in ENGINES.items():
        futures.append(ex.submit(_predict_one, sname, eng))
    for fut in tqdm(as_completed(futures), total=len(futures), desc="Predicting"):
        try:
            label_dir = fut.result()
            predict_jobs.append((Path(label_dir).name, label_dir))
            print(f"✅ {label_dir}")
        except Exception as e:
            print("❌ Predict failed:", e)

print(f"🎯 총 {len(predict_jobs)}개 시드 예측 완료")
if not predict_jobs:
    raise SystemExit("No predictions were produced. Check engines/paths.")

# ==============================
# 2) 글로벌 앙상블 (강신뢰 기능 적용)
# ==============================
def ensemble_merge(label_dirs, out_dir: Path,
                   iou_thr=IOU_MERGE, min_match=MIN_MATCH, conf_single=CONF_THRESH_SINGLE):
    out_dir.mkdir(parents=True, exist_ok=True)
    base = label_dirs[0]
    all_files = sorted(os.listdir(base))

    for file in tqdm(all_files, desc=f"Ensembling → {out_dir.name}"):
        preds_all = [load_yolo_txt(Path(d) / file) for d in label_dirs]
        combined = np.concatenate(preds_all, axis=0) if len(preds_all) else np.zeros((0, 6), dtype=np.float32)
        if len(combined) == 0:
            open(out_dir / file, "w").close()
            continue

        boxes_xyxy = xywh2xyxy(combined[:, 1:5])
        cls = combined[:, 0].astype(int)
        conf = combined[:, 5].astype(np.float32)

        used = np.zeros(len(combined), dtype=bool)
        final_preds = []

        for i in range(len(combined)):
            if used[i]:
                continue
            same_cls_idx = np.where(cls == cls[i])[0]
            ious = box_iou_np(boxes_xyxy[i:i+1], boxes_xyxy[same_cls_idx])[0]
            matched = same_cls_idx[ious >= iou_thr]
            used[matched] = True

            k = len(matched)
            if k >= min_match:
                avg_box = boxes_xyxy[matched].mean(axis=0)

                # ✅ 강신뢰 규칙 적용
                if np.any(conf[matched] >= CONF_HIGH_STRONG):
                    avg_conf = conf[matched].max()
                else:
                    avg_conf = conf[matched].mean()

                final_preds.append([cls[i], *avg_box, avg_conf])
            elif k == 1 and conf[i] >= conf_single:
                final_preds.append([cls[i], *boxes_xyxy[i], conf[i]])

        if len(final_preds):
            final_preds = np.array(final_preds, dtype=np.float32)
            xywh = xyxy2xywh(final_preds[:, 1:5])
            out = np.column_stack([final_preds[:, 0], xywh, final_preds[:, 5]])
            np.savetxt(out_dir / file, out, fmt="%.6f")
        else:
            open(out_dir / file, "w").close()

ALL_LABEL_DIRS = [ld for _, ld in predict_jobs]
ALL_ENSEMBLE_DIR = BASE_DIR / "all_ensembles" / "labels"
ensemble_merge(ALL_LABEL_DIRS, ALL_ENSEMBLE_DIR)
print(f"✅ Global ensemble saved: {ALL_ENSEMBLE_DIR}")

# ==============================
# 3) mAP clone(val@0.001) + AUROC + Divot ROC
# ==============================
def evaluate_map_auroc(PRED_LABEL_DIR: Path, title: str):
    print(f"\n================= ✅ EVALUATION — {title} =================")
    gt_files = sorted(GT_LABEL_DIR.glob("*.txt"))
    pred_map = {p.stem: p for p in PRED_LABEL_DIR.glob("*.txt")}

    # ----- mAP clone -----
    tp_list, conf_list, pred_cls_list, target_cls_list = [], [], [], []
    for gf in tqdm(gt_files, desc="Collecting stats for mAP..."):
        stem = gf.stem
        pf = pred_map.get(stem, None)

        preds = load_yolo_txt(pf)
        gts   = load_yolo_txt(gf)

        gt_boxes   = xywh2xyxy(gts[:, 1:5]) if len(gts) else np.zeros((0, 4), dtype=np.float32)
        gt_cls     = gts[:, 0].astype(int)  if len(gts) else np.zeros((0,), dtype=int)
        pred_boxes = xywh2xyxy(preds[:, 1:5]) if len(preds) else np.zeros((0, 4), dtype=np.float32)
        pred_cls   = preds[:, 0].astype(int) if len(preds) else np.zeros((0,), dtype=int)
        conf       = preds[:, 5].astype(np.float32) if len(preds) else np.zeros((0,), dtype=np.float32)

        correct = np.zeros((len(preds), len(IOU_THRS)), dtype=bool)
        if len(preds) and len(gts):
            ious = box_iou_np(pred_boxes, gt_boxes)
            order = stable_conf_sort(conf)
            pred_boxes, pred_cls, conf = pred_boxes[order], pred_cls[order], conf[order]
            ious = ious[order]
            for j, thr in enumerate(IOU_THRS):
                detected = []
                for k, pc in enumerate(pred_cls):
                    best_iou, best_gt = 0.0, -1
                    for gi, gc in enumerate(gt_cls):
                        if gc != pc or gi in detected:
                            continue
                        iou_val = ious[k, gi]
                        if iou_val > best_iou:
                            best_iou, best_gt = iou_val, gi
                    if best_iou > thr:   # val과 동일: >
                        correct[k, j] = True
                        detected.append(best_gt)

        tp_list.append(correct)
        conf_list.append(conf)
        pred_cls_list.append(pred_cls)
        target_cls_list.append(gt_cls)

    tp             = cat_or_empty(tp_list, axis=0, empty_shape=(0, len(IOU_THRS)), dtype=bool)
    conf_all       = cat_or_empty(conf_list, axis=0, empty_shape=(0,), dtype=np.float32)
    pred_cls_all   = cat_or_empty(pred_cls_list, axis=0, empty_shape=(0,), dtype=int)
    target_cls_all = cat_or_empty(target_cls_list, axis=0, empty_shape=(0,), dtype=int)

    p, r, ap, f1, ap_class = ap_per_class_compat(tp, conf_all, pred_cls_all, target_cls_all)

    # 안전 캐스팅
    p_arr = np.asarray(p, dtype=float).reshape(-1)
    r_arr = np.asarray(r, dtype=float).reshape(-1)
    ap_np = np.asarray(ap, dtype=float)
    if ap_np.ndim == 1:
        n_iou = len(IOU_THRS)
        assert ap_np.size % n_iou == 0, "ap shape unexpected; cannot reshape"
        ap_np = ap_np.reshape(-1, n_iou)

    mAP50   = float(ap_np[:, 0].mean()) if ap_np.size else 0.0
    mAP5095 = float(ap_np.mean()) if ap_np.size else 0.0
    ap_dict = {
        int(c): (float(p_arr[i]) if i < p_arr.size else 0.0,
                 float(r_arr[i]) if i < r_arr.size else 0.0,
                 float(ap_np[i, 0]) if i < ap_np.shape[0] else 0.0,
                 float(ap_np[i].mean()) if i < ap_np.shape[0] else 0.0)
        for i, c in enumerate(ap_class)
    }

    print(f"{'all':<15}  mP={p_arr.mean():.3f}  mR={r_arr.mean():.3f}  mAP50={mAP50:.3f}  mAP50-95={mAP5095:.3f}")
    for i in range(NUM_CLASSES):
        if i in ap_dict:
            pi, ri, ap50_i, ap5095_i = ap_dict[i]
        else:
            pi, ri, ap50_i, ap5095_i = 0.0, 0.0, 0.0, 0.0
        print(f"{CLASS_NAMES[i]:<15}  P={pi:.3f}  R={ri:.3f}  mAP50={ap50_i:.3f}  mAP50-95={ap5095_i:.3f}")

    # ----- AUROC (FN을 점수 0으로 반영) -----
    per_true  = {i: [] for i in range(NUM_CLASSES)}
    per_score = {i: [] for i in range(NUM_CLASSES)}
    tp_cnt = np.zeros(NUM_CLASSES, dtype=int)
    fp_cnt = np.zeros(NUM_CLASSES, dtype=int)
    fn_cnt = np.zeros(NUM_CLASSES, dtype=int)
    gt_cnt = np.zeros(NUM_CLASSES, dtype=int)

    for gf in tqdm(gt_files, desc="Matching for AUROC..."):
        stem = gf.stem
        pf = pred_map.get(stem, None)

        preds = load_yolo_txt(pf)
        gts   = load_yolo_txt(gf)

        if len(gts):
            for c in gts[:, 0].astype(int):
                gt_cnt[c] += 1

        if len(preds) == 0 and len(gts) > 0:
            for c in gts[:, 0].astype(int):
                fn_cnt[c] += 1
                per_true[c].append(1)
                per_score[c].append(0.0)
            continue

        gt_boxes   = xywh2xyxy(gts[:, 1:5]) if len(gts) else np.zeros((0, 4), dtype=np.float32)
        gt_cls     = gts[:, 0].astype(int)  if len(gts) else np.zeros((0,), dtype=int)
        pred_boxes = xywh2xyxy(preds[:, 1:5]) if len(preds) else np.zeros((0, 4), dtype=np.float32)
        pred_cls   = preds[:, 0].astype(int) if len(preds) else np.zeros((0,), dtype=int)
        conf       = preds[:, 5].astype(np.float32) if len(preds) else np.zeros((0,), dtype=np.float32)

        if len(preds) and len(gts):
            ious = box_iou_np(pred_boxes, gt_boxes)
            order = stable_conf_sort(conf)
            pred_boxes, pred_cls, conf = pred_boxes[order], pred_cls[order], conf[order]
            ious = ious[order]

            used = set()
            for k, (pc, cf) in enumerate(zip(pred_cls, conf)):
                best_iou, best_gt = 0.0, -1
                for gi, gc in enumerate(gt_cls):
                    if gc != pc or gi in used:
                        continue
                    iou_val = ious[k, gi]
                    if iou_val > best_iou:
                        best_iou, best_gt = iou_val, gi
                if best_iou > MATCH_IOU and best_gt != -1:
                    tp_cnt[pc] += 1
                    per_true[pc].append(1)
                    per_score[pc].append(cf)
                    used.add(best_gt)
                else:
                    fp_cnt[pc] += 1
                    per_true[pc].append(0)
                    per_score[pc].append(cf)

            for gi, gc in enumerate(gt_cls):
                if gi not in used:
                    fn_cnt[gc] += 1
                    per_true[gc].append(1)
                    per_score[gc].append(0.0)
        elif len(preds) > 0 and len(gts) == 0:
            for c, cf in zip(pred_cls, conf):
                fp_cnt[c] += 1
                per_true[c].append(0)
                per_score[c].append(cf)

    # ── Divot 전용 ROC 저장 ──
    if DRAW_DIVOT_ROC and TARGET_CLASS_NAME in CLASS_NAMES:
        divot_id = CLASS_NAMES.index(TARGET_CLASS_NAME)
        y_true  = np.asarray(per_true[divot_id], dtype=np.float32)
        y_score = np.asarray(per_score[divot_id], dtype=np.float32)

        roc_dir = BASE_DIR / ROC_OUTDIR_NAME
        roc_dir.mkdir(parents=True, exist_ok=True)

        # ROC 곡선
        if len(y_true) >= 2 and len(np.unique(y_true)) >= 2:
            fpr, tpr, _ = roc_curve(y_true, y_score)
            roc_auc = auc(fpr, tpr)
        else:
            fpr, tpr, roc_auc = np.array([0,1]), np.array([0,1]), float("nan")

        plt.figure(figsize=(5,5))
        plt.plot(fpr, tpr, marker='.', linewidth=1, label=f'ROC (AUC={roc_auc:.4f})')
        plt.plot([0,1],[0,1],'--', linewidth=1, label='chance')
        plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title(f'ROC — {TARGET_CLASS_NAME}')
        plt.legend(); plt.grid(True, alpha=0.3)
        fig_path = roc_dir / "divot_roc_curve.png"
        plt.savefig(fig_path, bbox_inches="tight", dpi=150)
        plt.close()
        print(f"[ROC] Divot ROC curve saved: {fig_path}")

        # 임계값 스윕 표
        rows = []
        P = int((y_true == 1).sum())
        N = int((y_true == 0).sum())
        for thr in ROC_THRESH_GRID:
            y_pred = (y_score >= thr).astype(np.int32)
            TP = int(((y_pred == 1) & (y_true == 1)).sum())
            FP = int(((y_pred == 1) & (y_true == 0)).sum())
            TN = int(((y_pred == 0) & (y_true == 0)).sum())
            FN = int(((y_pred == 0) & (y_true == 1)).sum())
            TPR = (TP / P) if P > 0 else 0.0
            FPR = (FP / N) if N > 0 else 0.0
            rows.append((thr, TP, FP, TN, FN, TPR, FPR))
        rows.sort(key=lambda x: x[0])

        csv_path = roc_dir / "divot_roc_table.csv"
        with open(csv_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["threshold", "TP", "FP", "TN", "FN", "TPR", "FPR"])
            writer.writerows(rows)
        print(f"[ROC] Divot ROC table saved: {csv_path}")

        # 콘솔 프리뷰
        print("\n[ROC] Divot — threshold sweep preview")
        print("thr     TP    FP    TN    FN    TPR      FPR")
        for r in rows[:11]:
            print(f"{r[0]:.3f}  {r[1]:5d} {r[2]:5d} {r[3]:5d} {r[4]:5d}  {r[5]:.4f}  {r[6]:.4f}")

    # ── AUROC 표준 출력 ──
    print("\n📈 AUROC Evaluation per class")
    print("─────────────────────────────────────────────────────────────")
    aurocs = []
    for i in range(NUM_CLASSES):
        y_true_i  = np.asarray(per_true[i], dtype=np.float32)
        y_score_i = np.asarray(per_score[i], dtype=np.float32)
        if len(y_true_i) < 2 or len(np.unique(y_true_i)) < 2:
            auroc_i = float("nan")
        else:
            try:
                auroc_i = roc_auc_score(y_true_i, y_score_i)
            except ValueError:
                auroc_i = float("nan")
        aurocs.append(auroc_i)
        auc_str = f"{auroc_i:.4f}" if not math.isnan(auroc_i) else "NaN"
        print(f"{CLASS_NAMES[i]:<18s}: AUROC={auc_str:<8s} | "
              f"TP={tp_cnt[i]:<4d} FP={fp_cnt[i]:<4d} FN={fn_cnt[i]:<4d} GT={gt_cnt[i]:<4d}")
    print("─────────────────────────────────────────────────────────────")
    mean_auroc = np.nanmean(aurocs) if np.any(~np.isnan(aurocs)) else float("nan")
    if not math.isnan(mean_auroc):
        print(f"🔹 Mean AUROC (macro): {mean_auroc:.4f}")
    else:
        print("🔹 Mean AUROC: NaN")
    print("======================================================================")

# 실행 (글로벌만)
evaluate_map_auroc(ALL_ENSEMBLE_DIR, title=f"Global Ensemble — ALL SEEDS ({RUN_NAME})")

# ==============================
# 4) (옵션) 앙상블 시각화/저장 — GT/TP/FP 구분 + Divot 전용 모드
# ==============================
if VIZ_SAVE:
    viz_dir = BASE_DIR / "all_ensembles" / VIZ_OUTDIR_NAME
    viz_dir.mkdir(parents=True, exist_ok=True)
    thr = PRED_CONF if VIZ_USE_PRED_CONF else VIZ_SCORE_MIN

    print(f"\n🖼  Drawing ENSEMBLE predictions (GT=green, TP=blue, FP=red)"
          f"{' [DIVOT ONLY]' if VIZ_DIVOT_ONLY else ''} → {viz_dir} (thr={thr})")

    img_paths = [p for p in sorted(TEST_DIR.glob("*"))
                 if p.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}]

    def iou(box1, box2):
        x1_min, y1_min = box1[0]-box1[2]/2, box1[1]-box1[3]/2
        x1_max, y1_max = box1[0]+box1[2]/2, box1[1]+box1[3]/2
        x2_min, y2_min = box2[0]-box2[2]/2, box2[1]-box2[3]/2
        x2_max, y2_max = box2[0]+box2[2]/2, box2[1]+box2[3]/2
        inter_w = max(0, min(x1_max, x2_max) - max(x1_min, x2_min))
        inter_h = max(0, min(y1_max, y2_max) - max(y1_min, y2_min))
        inter_area = inter_w * inter_h
        union_area = box1[2]*box1[3] + box2[2]*box2[3] - inter_area
        return inter_area / union_area if union_area > 0 else 0.0

    # Divot 클래스 ID 찾기
    divot_id = None
    if "Divot" in CLASS_NAMES:
        divot_id = CLASS_NAMES.index("Divot")

    for img_path in tqdm(img_paths, desc="Drawing (GT/TP/FP)"):
        stem = img_path.stem
        gt_path = GT_LABEL_DIR / f"{stem}.txt"
        pred_path = ALL_ENSEMBLE_DIR / f"{stem}.txt"
        if not gt_path.exists() and not pred_path.exists():
            continue

        img = cv2.imread(str(img_path))
        if img is None:
            continue
        H, W = img.shape[:2]
        thick = max(1, int(VIZ_THICK_BASE * (W + H) / 2000))

        # ── GT 로드 ──
        gt_boxes = []
        if gt_path.exists():
            with open(gt_path, "r") as f:
                for line in f:
                    vals = list(map(float, line.strip().split()))
                    if len(vals) >= 5:
                        cls = int(vals[0])
                        if VIZ_DIVOT_ONLY and cls != divot_id:
                            continue
                        gt_boxes.append((cls, vals[1:5]))

        # ── 예측 로드 ──
        preds = []
        if pred_path.exists():
            with open(pred_path, "r") as f:
                for line in f:
                    vals = list(map(float, line.strip().split()))
                    if len(vals) >= 6:
                        cls, x, y, w, h, conf = vals
                        if conf < thr:
                            continue
                        if VIZ_DIVOT_ONLY and int(cls) != divot_id:
                            continue
                        preds.append((int(cls), [x, y, w, h], conf))

        matched_gts = set()

        # ── 예측 박스: TP(파랑)/FP(빨강) ──
        for cls, box, conf in preds:
            matched = False
            for gi, (g_cls, g_box) in enumerate(gt_boxes):
                if g_cls != cls or gi in matched_gts:
                    continue
                if iou(box, g_box) > MATCH_IOU:
                    matched = True
                    matched_gts.add(gi)
                    break

            cx, cy, bw, bh = box
            x1, y1, x2, y2 = int((cx-bw/2)*W), int((cy-bh/2)*H), int((cx+bw/2)*W), int((cy+bh/2)*H)
            if matched:
                color, label = (255, 0, 0), f"TP-{CLASS_NAMES[cls]} {conf:.2f}"   # 파랑
            else:
                color, label = (0, 0, 255), f"FP-{CLASS_NAMES[cls]} {conf:.2f}"   # 빨강
            cv2.rectangle(img, (x1, y1), (x2, y2), color, thick)
            cv2.putText(img, label, (x1, max(y1-5, 10)), cv2.FONT_HERSHEY_SIMPLEX,
                        VIZ_FONT_SCALE, color, VIZ_TEXT_THICK, cv2.LINE_AA)

        # ── GT 박스: 항상 표시 (TP/FN 구분 색상 다르게)
        for gi, (cls, box) in enumerate(gt_boxes):
            cx, cy, bw, bh = box
            x1, y1, x2, y2 = int((cx-bw/2)*W), int((cy-bh/2)*H), int((cx+bw/2)*W), int((cy+bh/2)*H)

            if gi in matched_gts:
                # TP에 해당하는 GT → 연한 초록색
                color, label = (0, 200, 0), f"GT(TP)-{CLASS_NAMES[cls]}"
            else:
                # FN (미검출 GT) → 진한 초록색
                color, label = (0, 255, 0), f"GT(FN)-{CLASS_NAMES[cls]}"

            cv2.rectangle(img, (x1, y1), (x2, y2), color, thick)
            cv2.putText(img, label, (x1, max(y1-5, 10)), cv2.FONT_HERSHEY_SIMPLEX,
                        VIZ_FONT_SCALE, color, VIZ_TEXT_THICK, cv2.LINE_AA)

        cv2.imwrite(str(viz_dir / img_path.name), img)


    print(f"✅ VIZ 완료 — 저장 경로: {viz_dir}")


### 앙상블 추가

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# ============================================================
# YOLOv8 TensorRT — Multi-Seed (No Groups)
# Parallel Predict → Global Ensemble → mAP (val@0.001 clone) & AUROC
# - 실행마다 고유 폴더 생성
# - per-image 로그 비표시
# - FN(미검출)을 AUROC에 점수 0 양성으로 반영
# - mAP clone과 동일하게 IoU 비교는 '>' 사용
# - (옵션) 앙상블 결과를 원본 이미지에 시각화/저장 (전 클래스)
# - (추가) Divot 전용 ROC 표/그래프 저장
# ============================================================

# ── [무음 설정] ultralytics 임포트 전에 ──
import os, logging, math, gc, random, csv
os.environ["ULTRALYTICS_QUIET"] = "1"   # Ultralytics 내부 로그 억제
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True,max_split_size_mb:128")

from concurrent.futures import ThreadPoolExecutor, as_completed
from ultralytics import YOLO
from ultralytics.utils import LOGGER
from ultralytics.utils.metrics import ap_per_class as _ap_per_class
from pathlib import Path
from tqdm import tqdm
import numpy as np
import torch
import yaml
from datetime import datetime
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import cv2  # ← 시각화용

# ── Ultralytics 로거 레벨 하향 ──
LOGGER.setLevel(logging.ERROR)
torch.set_num_threads(1)

# ==============================
# 공통 설정
# ==============================
TEST_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251110/images/test")
GT_LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251110/labels/test")
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251110/data.yaml"

# ---- 실행마다 고유 폴더 생성 ----
BASE_ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/runs/predict_eval/multimodel_ensemble_eval_no_groups")
RUN_NAME = f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
BASE_DIR = BASE_ROOT / RUN_NAME
BASE_DIR.mkdir(parents=True, exist_ok=True)

# 입력 해상도(정규화 복원용) — (주의) 시각화는 실제 이미지 W,H를 사용함
IMG_W, IMG_H = 1920, 1536

# Predict 설정
IMG_SIZE       = 640
DEVICE         = 0
PRED_CONF      = 0.001     # 저장용 conf (mAP clone은 conf 컷 없음)
PRED_IOU_NMS   = 0.5
AUGMENT        = False
AGNOSTIC_NMS   = True
RECT           = False
MAX_WORKERS    = 4

# Ensemble 설정
IOU_MERGE          = 0.7   # IoU ≥ IOU_MERGE → 동일 객체 간주
CONF_THRESH_SINGLE = 0.5  # 단일 매칭 채택 하한
MIN_MATCH          = 2     # 매칭된 시드 수

# Evaluation 설정
IOU_THRS = np.linspace(0.5, 0.95, 10, dtype=np.float32)  # mAP@0.5:0.95
MATCH_IOU = 0.5   # AUROC/TPFPFN 매칭 임계치 (mAP clone과 동일하게 '>')

# ===[시각화 토글/옵션 (앙상블)]====================================
VIZ_SAVE           = True         # ← True면 앙상블 결과를 원본 이미지에 그려 저장
VIZ_USE_PRED_CONF  = False           # ← True면 시각화 임계값을 PRED_CONF와 동일하게 사용
VIZ_SCORE_MIN      = 0.001           # ← VIZ_USE_PRED_CONF=False일 때 쓰는 별도 임계값
VIZ_OUTDIR_NAME    = "viz"          # 출력 폴더명 (all_ensembles/ 아래에 생성)
VIZ_DIVOT_ONLY     = True      # ✅ True면 Divot만 시각화, False면 전 클래스 표시
# 스타일
VIZ_THICK_BASE     = 2
VIZ_FONT_SCALE     = 0.5
VIZ_TEXT_THICK     = 1

# ===[Divot 전용 ROC 테이블/그래프]===============================
DRAW_DIVOT_ROC     = True
TARGET_CLASS_NAME  = "Divot"
# 기존: ROC_THRESH_GRID = np.linspace(0.0, 1.0, 11)
# 수정 ↓↓↓
ROC_THRESH_GRID    = np.concatenate(([0.001], np.linspace(0.0, 1.0, 11)))
ROC_OUTDIR_NAME    = "roc_divot"
# ================================================================

# ==============================
# 시드별 엔진 (그룹 없음)
# ==============================
ENGINES = {
    "best1": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/best_fp16_bs1_640px_static.engine",
    "best2": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/precision_BEST_epoch70_0.95995_fp16_bs1_640px_static.engine",
    "best3": "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/recall_BEST_epoch35_0.74789_fp16_bs1_640px_static.engine",
}
# ==============================
# 유틸
# ==============================
def load_yolo_txt(path: Path):
    if (path is None) or (not path.exists()):
        return np.zeros((0, 6), dtype=np.float32)
    rows = []
    with open(path, "r") as f:
        for line in f:
            vals = list(map(float, line.strip().split()))
            if len(vals) == 5:  # GT
                c, x, y, w, h = vals; conf = 1.0
            else:               # Pred
                c, x, y, w, h, conf = vals
            rows.append([c, x, y, w, h, conf])
    return np.asarray(rows, dtype=np.float32)

def xywh2xyxy(boxes, W=IMG_W, H=IMG_H):
    if len(boxes) == 0: return np.zeros((0, 4), dtype=np.float32)
    x, y, w, h = boxes.T
    x1 = (x - w/2) * W; y1 = (y - h/2) * H
    x2 = (x + w/2) * W; y2 = (y + h/2) * H
    return np.stack([x1, y1, x2, y2], axis=1).astype(np.float32)

def xyxy2xywh(boxes, W=IMG_W, H=IMG_H):
    if len(boxes) == 0: return np.zeros((0, 4), dtype=np.float32)
    x1, y1, x2, y2 = boxes.T
    x = (x1 + x2) / 2 / W; y = (y1 + y2) / 2 / H
    w = (x2 - x1) / W;     h = (y2 - y1) / H
    return np.stack([x, y, w, h], axis=1).astype(np.float32)

def box_iou_np(a, b):
    if len(a) == 0 or len(b) == 0:
        return np.zeros((len(a), len(b)), dtype=np.float32)
    area1 = (a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1])
    area2 = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
    ix1 = np.maximum(a[:, None, 0], b[:, 0]); iy1 = np.maximum(a[:, None, 1], b[:, 1])
    ix2 = np.minimum(a[:, None, 2], b[:, 2]); iy2 = np.minimum(a[:, None, 3], b[:, 3])
    iw = np.clip(ix2 - ix1, 0, None); ih = np.clip(iy2 - iy1, 0, None)
    inter = iw * ih
    return inter / (area1[:, None] + area2 - inter + 1e-6)

def stable_conf_sort(conf: np.ndarray):
    idx = np.arange(len(conf))
    return np.lexsort((idx, -conf))

def cat_or_empty(arrs, axis=0, empty_shape=None, dtype=None):
    arrs = [a for a in arrs if a is not None]
    if len(arrs) == 0:
        return np.zeros(empty_shape, dtype=dtype)
    try:
        return np.concatenate(arrs, axis=axis)
    except Exception:
        return np.zeros(empty_shape, dtype=dtype)

def ap_per_class_compat(tp, conf, pred_cls, target_cls):
    out = _ap_per_class(tp, conf, pred_cls, target_cls)
    if len(out) == 5:
        p, r, ap, f1, ap_class = out
    elif len(out) >= 7:
        p, r, ap, f1, ap_class = out[2], out[3], out[5], out[4], out[6]
    else:
        raise RuntimeError("Unexpected ap_per_class return format")
    return p, r, ap, f1, ap_class

def xywhn2xyxy_pixels(xywhn, W, H):
    if len(xywhn) == 0:
        return np.zeros((0, 4), dtype=np.float32)
    x, y, w, h = xywhn.T
    x1 = (x - w/2) * W
    y1 = (y - h/2) * H
    x2 = (x + w/2) * W
    y2 = (y + h/2) * H
    return np.stack([x1, y1, x2, y2], axis=1).astype(np.float32)

def _class_colors(n):
    colors = []
    for i in range(n):
        random.seed(i + 12345)
        colors.append((random.randint(60,255), random.randint(60,255), random.randint(60,255)))
    return colors

# ==============================
# 클래스 로드
# ==============================
with open(DATA_YAML, "r") as f:
    data_cfg = yaml.safe_load(f)
CLASS_NAMES = data_cfg["names"]
NUM_CLASSES = len(CLASS_NAMES)

# ==============================
# 1) Predict (스레드 병렬, 기본=순차)
# ==============================
print(f"🚀 Parallel inference for seeds (no groups) ... → {BASE_DIR}")
predict_jobs = []  # (seed, out_label_dir)

def _predict_one(seed_name, engine_path):
    out_dir = BASE_DIR / "raw_preds" / seed_name
    out_dir.mkdir(parents=True, exist_ok=True)
    model = YOLO(engine_path)
    model.predict(
        source=str(TEST_DIR),
        imgsz=IMG_SIZE,
        conf=PRED_CONF,
        iou=PRED_IOU_NMS,
        device=DEVICE,
        rect=RECT,
        agnostic_nms=AGNOSTIC_NMS,
        augment=AUGMENT,
        deterministic=True,
        seed=1,
        save_txt=True,
        save_conf=True,
        project=out_dir,
        name="pred_labels",
        exist_ok=True,
        verbose=False,
    )
    try:
        model.predictor = None
    except Exception:
        pass
    del model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.synchronize(); torch.cuda.empty_cache()
    return out_dir / "pred_labels" / "labels"

futures = []
with ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(ENGINES))) as ex:
    for sname, eng in ENGINES.items():
        futures.append(ex.submit(_predict_one, sname, eng))
    for fut in tqdm(as_completed(futures), total=len(futures), desc="Predicting"):
        try:
            label_dir = fut.result()
            predict_jobs.append((Path(label_dir).name, label_dir))
            print(f"✅ {label_dir}")
        except Exception as e:
            print("❌ Predict failed:", e)

print(f"🎯 총 {len(predict_jobs)}개 시드 예측 완료")
if not predict_jobs:
    raise SystemExit("No predictions were produced. Check engines/paths.")

# ==============================
# 2) 글로벌 앙상블 (모든 시드 합침)
# ==============================
def ensemble_merge(label_dirs, out_dir: Path,
                   iou_thr=IOU_MERGE, min_match=MIN_MATCH, conf_single=CONF_THRESH_SINGLE):
    out_dir.mkdir(parents=True, exist_ok=True)
    base = label_dirs[0]
    all_files = sorted(os.listdir(base))

    # (D) Adaptive IoU 설정
    BASE_IOU = iou_thr
    ADAPT_IOU_SCALE = 0.15

    def adaptive_iou_threshold(conf):
        return BASE_IOU + ADAPT_IOU_SCALE * conf

    # (C) Dynamic Rule 설정
    CONF_HIGH_STRONG = 0.85
    CONF_LOW_KEEP    = 0.30   # 약간 낮춰 recall 확보
    MIN_MATCH        = min_match

    # (🔥) Confidence 리스케일 함수 추가 — temperature scaling
    def rescale_conf(c):
        return 1 / (1 + np.exp(-8 * (c - 0.5)))

    for file in tqdm(all_files, desc=f"Ensembling → {out_dir.name}"):
        preds_all = [load_yolo_txt(Path(d) / file) for d in label_dirs]
        combined = np.concatenate(preds_all, axis=0) if len(preds_all) else np.zeros((0, 6), dtype=np.float32)
        if len(combined) == 0:
            open(out_dir / file, "w").close()
            continue

        boxes_xyxy = xywh2xyxy(combined[:, 1:5])
        cls = combined[:, 0].astype(int)
        conf = combined[:, 5].astype(np.float32)

        used = np.zeros(len(combined), dtype=bool)
        final_preds = []

        for i in range(len(combined)):
            if used[i]:
                continue

            same_cls_idx = np.where(cls == cls[i])[0]
            ious = box_iou_np(boxes_xyxy[i:i+1], boxes_xyxy[same_cls_idx])[0]

            # Adaptive IoU (평균 기반 + clip)
            dynamic_thr = adaptive_iou_threshold(np.mean(conf[same_cls_idx]))
            dynamic_thr = np.clip(dynamic_thr, 0.50, 0.70)
            matched = same_cls_idx[ious >= dynamic_thr]
            used[matched] = True

            k = len(matched)
            conf_group = conf[matched]
            avg_box = boxes_xyxy[matched].mean(axis=0)

            # Dynamic Rule
            if k >= MIN_MATCH or any(c >= CONF_HIGH_STRONG for c in conf_group):
                conf_out = conf_group.max()
            elif np.mean(conf_group) >= CONF_LOW_KEEP:
                conf_out = np.mean(conf_group)
            elif k == 1 and conf[i] >= conf_single:
                conf_out = conf[i]
            else:
                continue

            # (🔥) rescale 적용 — sigmoid-like scaling
            conf_out = float(rescale_conf(conf_out))

            final_preds.append([cls[i], *avg_box, conf_out])

        if len(final_preds):
            final_preds = np.array(final_preds, dtype=np.float32)
            xywh = xyxy2xywh(final_preds[:, 1:5])
            out = np.column_stack([final_preds[:, 0], xywh, final_preds[:, 5]])
            np.savetxt(out_dir / file, out, fmt="%.6f")
        else:
            open(out_dir / file, "w").close()

ALL_LABEL_DIRS = [ld for _, ld in predict_jobs]
ALL_ENSEMBLE_DIR = BASE_DIR / "all_ensembles" / "labels"
ensemble_merge(ALL_LABEL_DIRS, ALL_ENSEMBLE_DIR)
print(f"✅ Global ensemble saved: {ALL_ENSEMBLE_DIR}")

# ==============================
# 3) mAP clone(val@0.001) + AUROC + Divot ROC
# ==============================
def evaluate_map_auroc(PRED_LABEL_DIR: Path, title: str):
    print(f"\n================= ✅ EVALUATION — {title} =================")
    gt_files = sorted(GT_LABEL_DIR.glob("*.txt"))
    pred_map = {p.stem: p for p in PRED_LABEL_DIR.glob("*.txt")}

    # ----- mAP clone -----
    tp_list, conf_list, pred_cls_list, target_cls_list = [], [], [], []
    for gf in tqdm(gt_files, desc="Collecting stats for mAP..."):
        stem = gf.stem
        pf = pred_map.get(stem, None)

        preds = load_yolo_txt(pf)
        gts   = load_yolo_txt(gf)

        gt_boxes   = xywh2xyxy(gts[:, 1:5]) if len(gts) else np.zeros((0, 4), dtype=np.float32)
        gt_cls     = gts[:, 0].astype(int)  if len(gts) else np.zeros((0,), dtype=int)
        pred_boxes = xywh2xyxy(preds[:, 1:5]) if len(preds) else np.zeros((0, 4), dtype=np.float32)
        pred_cls   = preds[:, 0].astype(int) if len(preds) else np.zeros((0,), dtype=int)
        conf       = preds[:, 5].astype(np.float32) if len(preds) else np.zeros((0,), dtype=np.float32)

        correct = np.zeros((len(preds), len(IOU_THRS)), dtype=bool)
        if len(preds) and len(gts):
            ious = box_iou_np(pred_boxes, gt_boxes)
            order = stable_conf_sort(conf)
            pred_boxes, pred_cls, conf = pred_boxes[order], pred_cls[order], conf[order]
            ious = ious[order]
            for j, thr in enumerate(IOU_THRS):
                detected = []
                for k, pc in enumerate(pred_cls):
                    best_iou, best_gt = 0.0, -1
                    for gi, gc in enumerate(gt_cls):
                        if gc != pc or gi in detected:
                            continue
                        iou_val = ious[k, gi]
                        if iou_val > best_iou:
                            best_iou, best_gt = iou_val, gi
                    if best_iou > thr:   # val과 동일: >
                        correct[k, j] = True
                        detected.append(best_gt)

        tp_list.append(correct)
        conf_list.append(conf)
        pred_cls_list.append(pred_cls)
        target_cls_list.append(gt_cls)

    tp             = cat_or_empty(tp_list, axis=0, empty_shape=(0, len(IOU_THRS)), dtype=bool)
    conf_all       = cat_or_empty(conf_list, axis=0, empty_shape=(0,), dtype=np.float32)
    pred_cls_all   = cat_or_empty(pred_cls_list, axis=0, empty_shape=(0,), dtype=int)
    target_cls_all = cat_or_empty(target_cls_list, axis=0, empty_shape=(0,), dtype=int)

    p, r, ap, f1, ap_class = ap_per_class_compat(tp, conf_all, pred_cls_all, target_cls_all)

    # 안전 캐스팅
    p_arr = np.asarray(p, dtype=float).reshape(-1)
    r_arr = np.asarray(r, dtype=float).reshape(-1)
    ap_np = np.asarray(ap, dtype=float)
    if ap_np.ndim == 1:
        n_iou = len(IOU_THRS)
        assert ap_np.size % n_iou == 0, "ap shape unexpected; cannot reshape"
        ap_np = ap_np.reshape(-1, n_iou)

    mAP50   = float(ap_np[:, 0].mean()) if ap_np.size else 0.0
    mAP5095 = float(ap_np.mean()) if ap_np.size else 0.0
    ap_dict = {
        int(c): (float(p_arr[i]) if i < p_arr.size else 0.0,
                 float(r_arr[i]) if i < r_arr.size else 0.0,
                 float(ap_np[i, 0]) if i < ap_np.shape[0] else 0.0,
                 float(ap_np[i].mean()) if i < ap_np.shape[0] else 0.0)
        for i, c in enumerate(ap_class)
    }

    print(f"{'all':<15}  mP={p_arr.mean():.3f}  mR={r_arr.mean():.3f}  mAP50={mAP50:.3f}  mAP50-95={mAP5095:.3f}")
    for i in range(NUM_CLASSES):
        if i in ap_dict:
            pi, ri, ap50_i, ap5095_i = ap_dict[i]
        else:
            pi, ri, ap50_i, ap5095_i = 0.0, 0.0, 0.0, 0.0
        print(f"{CLASS_NAMES[i]:<15}  P={pi:.3f}  R={ri:.3f}  mAP50={ap50_i:.3f}  mAP50-95={ap5095_i:.3f}")

    # ----- AUROC (FN을 점수 0으로 반영) -----
    per_true  = {i: [] for i in range(NUM_CLASSES)}
    per_score = {i: [] for i in range(NUM_CLASSES)}
    tp_cnt = np.zeros(NUM_CLASSES, dtype=int)
    fp_cnt = np.zeros(NUM_CLASSES, dtype=int)
    fn_cnt = np.zeros(NUM_CLASSES, dtype=int)
    gt_cnt = np.zeros(NUM_CLASSES, dtype=int)

    for gf in tqdm(gt_files, desc="Matching for AUROC..."):
        stem = gf.stem
        pf = pred_map.get(stem, None)

        preds = load_yolo_txt(pf)
        gts   = load_yolo_txt(gf)

        if len(gts):
            for c in gts[:, 0].astype(int):
                gt_cnt[c] += 1

        if len(preds) == 0 and len(gts) > 0:
            for c in gts[:, 0].astype(int):
                fn_cnt[c] += 1
                per_true[c].append(1)
                per_score[c].append(0.0)
            continue

        gt_boxes   = xywh2xyxy(gts[:, 1:5]) if len(gts) else np.zeros((0, 4), dtype=np.float32)
        gt_cls     = gts[:, 0].astype(int)  if len(gts) else np.zeros((0,), dtype=int)
        pred_boxes = xywh2xyxy(preds[:, 1:5]) if len(preds) else np.zeros((0, 4), dtype=np.float32)
        pred_cls   = preds[:, 0].astype(int) if len(preds) else np.zeros((0,), dtype=int)
        conf       = preds[:, 5].astype(np.float32) if len(preds) else np.zeros((0,), dtype=np.float32)

        if len(preds) and len(gts):
            ious = box_iou_np(pred_boxes, gt_boxes)
            order = stable_conf_sort(conf)
            pred_boxes, pred_cls, conf = pred_boxes[order], pred_cls[order], conf[order]
            ious = ious[order]

            used = set()
            for k, (pc, cf) in enumerate(zip(pred_cls, conf)):
                best_iou, best_gt = 0.0, -1
                for gi, gc in enumerate(gt_cls):
                    if gc != pc or gi in used:
                        continue
                    iou_val = ious[k, gi]
                    if iou_val > best_iou:
                        best_iou, best_gt = iou_val, gi
                if best_iou > MATCH_IOU and best_gt != -1:
                    tp_cnt[pc] += 1
                    per_true[pc].append(1)
                    per_score[pc].append(cf)
                    used.add(best_gt)
                else:
                    fp_cnt[pc] += 1
                    per_true[pc].append(0)
                    per_score[pc].append(cf)

            for gi, gc in enumerate(gt_cls):
                if gi not in used:
                    fn_cnt[gc] += 1
                    per_true[gc].append(1)
                    per_score[gc].append(0.0)
        elif len(preds) > 0 and len(gts) == 0:
            for c, cf in zip(pred_cls, conf):
                fp_cnt[c] += 1
                per_true[c].append(0)
                per_score[c].append(cf)

    # ── Divot 전용 ROC 저장 ──
    if DRAW_DIVOT_ROC and TARGET_CLASS_NAME in CLASS_NAMES:
        divot_id = CLASS_NAMES.index(TARGET_CLASS_NAME)
        y_true  = np.asarray(per_true[divot_id], dtype=np.float32)
        y_score = np.asarray(per_score[divot_id], dtype=np.float32)

        roc_dir = BASE_DIR / ROC_OUTDIR_NAME
        roc_dir.mkdir(parents=True, exist_ok=True)

        # ROC 곡선
        if len(y_true) >= 2 and len(np.unique(y_true)) >= 2:
            fpr, tpr, _ = roc_curve(y_true, y_score)
            roc_auc = auc(fpr, tpr)
        else:
            fpr, tpr, roc_auc = np.array([0,1]), np.array([0,1]), float("nan")

        plt.figure(figsize=(5,5))
        plt.plot(fpr, tpr, marker='.', linewidth=1, label=f'ROC (AUC={roc_auc:.4f})')
        plt.plot([0,1],[0,1],'--', linewidth=1, label='chance')
        plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title(f'ROC — {TARGET_CLASS_NAME}')
        plt.legend(); plt.grid(True, alpha=0.3)
        fig_path = roc_dir / "divot_roc_curve.png"
        plt.savefig(fig_path, bbox_inches="tight", dpi=150)
        plt.close()
        print(f"[ROC] Divot ROC curve saved: {fig_path}")

        # 임계값 스윕 표
        rows = []
        P = int((y_true == 1).sum())
        N = int((y_true == 0).sum())
        for thr in ROC_THRESH_GRID:
            y_pred = (y_score >= thr).astype(np.int32)
            TP = int(((y_pred == 1) & (y_true == 1)).sum())
            FP = int(((y_pred == 1) & (y_true == 0)).sum())
            TN = int(((y_pred == 0) & (y_true == 0)).sum())
            FN = int(((y_pred == 0) & (y_true == 1)).sum())
            TPR = (TP / P) if P > 0 else 0.0
            FPR = (FP / N) if N > 0 else 0.0
            rows.append((thr, TP, FP, TN, FN, TPR, FPR))
        rows.sort(key=lambda x: x[0])

        csv_path = roc_dir / "divot_roc_table.csv"
        with open(csv_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["threshold", "TP", "FP", "TN", "FN", "TPR", "FPR"])
            writer.writerows(rows)
        print(f"[ROC] Divot ROC table saved: {csv_path}")

        # 콘솔 프리뷰
        print("\n[ROC] Divot — threshold sweep preview")
        print("thr     TP    FP    TN    FN    TPR      FPR")
        for r in rows[:11]:
            print(f"{r[0]:.3f}  {r[1]:5d} {r[2]:5d} {r[3]:5d} {r[4]:5d}  {r[5]:.4f}  {r[6]:.4f}")

    # ── AUROC 표준 출력 ──
    print("\n📈 AUROC Evaluation per class")
    print("─────────────────────────────────────────────────────────────")
    aurocs = []
    for i in range(NUM_CLASSES):
        y_true_i  = np.asarray(per_true[i], dtype=np.float32)
        y_score_i = np.asarray(per_score[i], dtype=np.float32)
        if len(y_true_i) < 2 or len(np.unique(y_true_i)) < 2:
            auroc_i = float("nan")
        else:
            try:
                auroc_i = roc_auc_score(y_true_i, y_score_i)
            except ValueError:
                auroc_i = float("nan")
        aurocs.append(auroc_i)
        auc_str = f"{auroc_i:.4f}" if not math.isnan(auroc_i) else "NaN"
        print(f"{CLASS_NAMES[i]:<18s}: AUROC={auc_str:<8s} | "
              f"TP={tp_cnt[i]:<4d} FP={fp_cnt[i]:<4d} FN={fn_cnt[i]:<4d} GT={gt_cnt[i]:<4d}")
    print("─────────────────────────────────────────────────────────────")
    mean_auroc = np.nanmean(aurocs) if np.any(~np.isnan(aurocs)) else float("nan")
    if not math.isnan(mean_auroc):
        print(f"🔹 Mean AUROC (macro): {mean_auroc:.4f}")
    else:
        print("🔹 Mean AUROC: NaN")
    print("======================================================================")

# 실행 (글로벌만)
evaluate_map_auroc(ALL_ENSEMBLE_DIR, title=f"Global Ensemble — ALL SEEDS ({RUN_NAME})")

# ==============================
# 4) (옵션) 앙상블 시각화/저장 — GT/TP/FP 구분 + Divot 전용 모드
# ==============================
if VIZ_SAVE:
    viz_dir = BASE_DIR / "all_ensembles" / VIZ_OUTDIR_NAME
    viz_dir.mkdir(parents=True, exist_ok=True)
    thr = PRED_CONF if VIZ_USE_PRED_CONF else VIZ_SCORE_MIN

    print(f"\n🖼  Drawing ENSEMBLE predictions (GT=green, TP=blue, FP=red)"
          f"{' [DIVOT ONLY]' if VIZ_DIVOT_ONLY else ''} → {viz_dir} (thr={thr})")

    img_paths = [p for p in sorted(TEST_DIR.glob("*"))
                 if p.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}]

    def iou(box1, box2):
        x1_min, y1_min = box1[0]-box1[2]/2, box1[1]-box1[3]/2
        x1_max, y1_max = box1[0]+box1[2]/2, box1[1]+box1[3]/2
        x2_min, y2_min = box2[0]-box2[2]/2, box2[1]-box2[3]/2
        x2_max, y2_max = box2[0]+box2[2]/2, box2[1]+box2[3]/2
        inter_w = max(0, min(x1_max, x2_max) - max(x1_min, x2_min))
        inter_h = max(0, min(y1_max, y2_max) - max(y1_min, y2_min))
        inter_area = inter_w * inter_h
        union_area = box1[2]*box1[3] + box2[2]*box2[3] - inter_area
        return inter_area / union_area if union_area > 0 else 0.0

    # Divot 클래스 ID 찾기
    divot_id = None
    if "Divot" in CLASS_NAMES:
        divot_id = CLASS_NAMES.index("Divot")

    for img_path in tqdm(img_paths, desc="Drawing (GT/TP/FP)"):
        stem = img_path.stem
        gt_path = GT_LABEL_DIR / f"{stem}.txt"
        pred_path = ALL_ENSEMBLE_DIR / f"{stem}.txt"
        if not gt_path.exists() and not pred_path.exists():
            continue

        img = cv2.imread(str(img_path))
        if img is None:
            continue
        H, W = img.shape[:2]
        thick = max(1, int(VIZ_THICK_BASE * (W + H) / 2000))

        # ── GT 로드 ──
        gt_boxes = []
        if gt_path.exists():
            with open(gt_path, "r") as f:
                for line in f:
                    vals = list(map(float, line.strip().split()))
                    if len(vals) >= 5:
                        cls = int(vals[0])
                        if VIZ_DIVOT_ONLY and cls != divot_id:
                            continue
                        gt_boxes.append((cls, vals[1:5]))

        # ── 예측 로드 ──
        preds = []
        if pred_path.exists():
            with open(pred_path, "r") as f:
                for line in f:
                    vals = list(map(float, line.strip().split()))
                    if len(vals) >= 6:
                        cls, x, y, w, h, conf = vals
                        if conf < thr:
                            continue
                        if VIZ_DIVOT_ONLY and int(cls) != divot_id:
                            continue
                        preds.append((int(cls), [x, y, w, h], conf))

        matched_gts = set()

        # ── 예측 박스: TP(파랑)/FP(빨강) ──
        for cls, box, conf in preds:
            matched = False
            for gi, (g_cls, g_box) in enumerate(gt_boxes):
                if g_cls != cls or gi in matched_gts:
                    continue
                if iou(box, g_box) > MATCH_IOU:
                    matched = True
                    matched_gts.add(gi)
                    break

            cx, cy, bw, bh = box
            x1, y1, x2, y2 = int((cx-bw/2)*W), int((cy-bh/2)*H), int((cx+bw/2)*W), int((cy+bh/2)*H)
            if matched:
                color, label = (255, 0, 0), f"TP-{CLASS_NAMES[cls]} {conf:.2f}"   # 파랑
            else:
                color, label = (0, 0, 255), f"FP-{CLASS_NAMES[cls]} {conf:.2f}"   # 빨강
            cv2.rectangle(img, (x1, y1), (x2, y2), color, thick)
            cv2.putText(img, label, (x1, max(y1-5, 10)), cv2.FONT_HERSHEY_SIMPLEX,
                        VIZ_FONT_SCALE, color, VIZ_TEXT_THICK, cv2.LINE_AA)

        # ── GT 박스: 항상 표시 (TP/FN 구분 색상 다르게)
        for gi, (cls, box) in enumerate(gt_boxes):
            cx, cy, bw, bh = box
            x1, y1, x2, y2 = int((cx-bw/2)*W), int((cy-bh/2)*H), int((cx+bw/2)*W), int((cy+bh/2)*H)

            if gi in matched_gts:
                # TP에 해당하는 GT → 연한 초록색
                color, label = (0, 200, 0), f"GT(TP)-{CLASS_NAMES[cls]}"
            else:
                # FN (미검출 GT) → 진한 초록색
                color, label = (0, 255, 0), f"GT(FN)-{CLASS_NAMES[cls]}"

            cv2.rectangle(img, (x1, y1), (x2, y2), color, thick)
            cv2.putText(img, label, (x1, max(y1-5, 10)), cv2.FONT_HERSHEY_SIMPLEX,
                        VIZ_FONT_SCALE, color, VIZ_TEXT_THICK, cv2.LINE_AA)

        cv2.imwrite(str(viz_dir / img_path.name), img)


    print(f"✅ VIZ 완료 — 저장 경로: {viz_dir}")
